In [1]:
import google.colab
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [3]:
data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

In [106]:
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

# train_ft = pd.read_csv(data_path + 'train_ft_97983.csv')
train_ft = pd.read_csv(data_path + 'train_132220_ver1.csv')



In [107]:
train_ft

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
132215,건축구조+마감재,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...
132216,마감재,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...
132217,마감재,"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘",모노롱브릭타일은 일반 시멘트벽돌과는 다르게 얇고 긴 모양의 타일 형태로 와이드 벽돌...
132218,마감재,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...


# 카테고리 기준 분류

In [108]:
train_ft['category'].value_counts()

마감재            29353
마감재+마감하자        8801
마감재+인테리어        7838
마감재+시공          7535
인테리어            7336
시공              6680
마감하자+마감재        5878
시공+마감재          5787
인테리어+마감재        5708
마감하자            5200
시공+마감하자         2842
인테리어+마감하자       2727
마감하자+인테리어       2468
시공+인테리어         2466
마감하자+시공         2268
인테리어+시공         2244
건축구조+마감재        2074
마감재+건축구조        1934
마감재+기타          1772
기타+마감재          1698
타 마감하자+마감재      1619
건축구조            1477
마감재+타 마감하자      1435
기타              1237
건축구조+마감하자        962
타 마감하자           931
건축구조+인테리어        913
건축구조+시공          884
타 마감하자+마감하자      819
기타+인테리어          812
기타+마감하자          786
기타+시공            700
타 마감하자+시공        698
타 마감하자+인테리어      686
마감하자+건축구조        614
인테리어+건축구조        613
시공+건축구조          604
인테리어+기타          556
마감하자+기타          514
시공+기타            493
마감하자+타 마감하자      447
시공+타 마감하자        411
인테리어+타 마감하자      407
기타+건축구조          192
타 마감하자+건축구조      188
건축구조+타 마감하자      177
건축구조+기타          164
타 마감하자+기타    

In [109]:
train_ft['qna'] = train_ft['question'].str.cat(train_ft['answer'], sep=' ')

In [110]:
train_ft['category'] = train_ft['category'].str.replace(' ','').str.split('+')

In [111]:
train_ft['category'].value_counts()

[마감재]            29353
[마감재, 마감하자]       8801
[마감재, 인테리어]       7838
[마감재, 시공]         7535
[인테리어]            7336
[시공]              6680
[마감하자, 마감재]       5878
[시공, 마감재]         5787
[인테리어, 마감재]       5708
[마감하자]            5200
[시공, 마감하자]        2842
[인테리어, 마감하자]      2727
[마감하자, 인테리어]      2468
[시공, 인테리어]        2466
[마감하자, 시공]        2268
[인테리어, 시공]        2244
[건축구조, 마감재]       2074
[마감재, 건축구조]       1934
[마감재, 기타]         1772
[기타, 마감재]         1698
[타마감하자, 마감재]      1619
[건축구조]            1477
[마감재, 타마감하자]      1435
[기타]              1237
[건축구조, 마감하자]       962
[타마감하자]            931
[건축구조, 인테리어]       913
[건축구조, 시공]         884
[타마감하자, 마감하자]      819
[기타, 인테리어]         812
[기타, 마감하자]         786
[기타, 시공]           700
[타마감하자, 시공]        698
[타마감하자, 인테리어]      686
[마감하자, 건축구조]       614
[인테리어, 건축구조]       613
[시공, 건축구조]         604
[인테리어, 기타]         556
[마감하자, 기타]         514
[시공, 기타]           493
[마감하자, 타마감하자]      447
[시공, 타마감하자]        411
[인테리어, 타마감하자]      407
[기타, 건축구조] 

In [112]:
for i in tqdm(range(len(train_ft['category']))):
    if len(train_ft['category'][i]) == 2 and train_ft['category'][i][0] == train_ft['category'][i][1]:
        train_ft['category'][i] = [train_ft['category'][i][0]]
        # print(train_ft['category'][i])

  0%|          | 0/132220 [00:00<?, ?it/s]

In [113]:
train_ft['category'].value_counts()

[마감재]            29353
[마감재, 마감하자]       8801
[마감재, 인테리어]       7838
[마감재, 시공]         7535
[인테리어]            7336
[시공]              6680
[마감하자, 마감재]       5878
[시공, 마감재]         5787
[인테리어, 마감재]       5708
[마감하자]            5200
[시공, 마감하자]        2842
[인테리어, 마감하자]      2727
[마감하자, 인테리어]      2468
[시공, 인테리어]        2466
[마감하자, 시공]        2268
[인테리어, 시공]        2244
[건축구조, 마감재]       2074
[마감재, 건축구조]       1934
[마감재, 기타]         1772
[기타, 마감재]         1698
[타마감하자, 마감재]      1619
[건축구조]            1477
[마감재, 타마감하자]      1435
[기타]              1237
[건축구조, 마감하자]       962
[타마감하자]            931
[건축구조, 인테리어]       913
[건축구조, 시공]         884
[타마감하자, 마감하자]      819
[기타, 인테리어]         812
[기타, 마감하자]         786
[기타, 시공]           700
[타마감하자, 시공]        698
[타마감하자, 인테리어]      686
[마감하자, 건축구조]       614
[인테리어, 건축구조]       613
[시공, 건축구조]         604
[인테리어, 기타]         556
[마감하자, 기타]         514
[시공, 기타]           493
[마감하자, 타마감하자]      447
[시공, 타마감하자]        411
[인테리어, 타마감하자]      407
[기타, 건축구조] 

In [114]:
len(train_ft['category'].value_counts())

49

In [115]:
train_ft

,category,question,answer,qna
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사..."
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...,...
132215,"[건축구조, 마감재]","경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ..."
132216,[마감재],"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나..."
132217,[마감재],"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘",모노롱브릭타일은 일반 시멘트벽돌과는 다르게 얇고 긴 모양의 타일 형태로 와이드 벽돌...,"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘 모노롱브릭타일은 일반 시멘트벽돌과..."
132218,[마감재],"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로..."


# 질문 유형 분류

In [116]:
train_ft['qna']

0         면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...
1         내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.
2         철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...
3         철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사...
4              철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.
                                ...                        
132215    경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ...
132216    아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나...
132217    롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘 모노롱브릭타일은 일반 시멘트벽돌과...
132218    셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로...
132219    실내 공간에 자연적인 감각을 불어넣기 위한 방법이 무엇이며, 배관공사를 할 때 주의...
Name: qna, Length: 132220, dtype: object

In [117]:
train_ft['question'].tolist()[-1000:]


# 설명 - 뭐야?, 설명해줘, 알려줘,
# 종류
# 장점, 단점
# 기능 - 효과,
# 용도 - 어디에 사용,
# 관계 - 차이점, 차이,
# 비교 - 뭐가 더,
# 원인 - 문제의 원인
# 사용 이유 - 이유, 주의할 이유
# 구조
# 방식
# 방법, 법, - 어디에 사용, 늘리는 법, 어떻게 구분,
# 어디가 안좋아? / 어떤 점이 안좋아?
# 해결 - 해결, 어떻게, 처리, 대책
# 계산 - 평수, 측정, 지점,
# 수명, 기간 - 얼마나,
# 때 - 언제 발생,
# 판단 - 고려할 요소, 판정, 기준
# 추천 - 적합할까?, 중 어느 것이 좋아?, 뭐가 있어?, 인기, 어떤 재료, 어떤 것, 선택
# 주의 - 주의할 점
# 팁 - 꿀팁, 팁, 아이디어

# 하자 - 손상, 찢어졌, 하자,

['모던한 인테리어 스타일을 구성하는 핵심 포인트는 무엇인지, 또한, 강화마루를 사용할 때 주의해야 하는 점 알려줘',
 '새집증후군을 예방하는 방법이 무엇인지, 그리고 전벽돌이 뭐야?',
 '천연페인트의 특징이 무엇인지, 또한, 노출콘크리트가 뭐야?',
 '토목이 무엇이며, 부실 시공으로 인해 타공하자가 발생할 수 있어?',
 '알루미늄 샤시의 장점이 무엇이며, 아크릴 계열의 특징 알려줘',
 '인테리어에서 벽지를 선택하는데 주의해야 할 점은 무엇인지, 그리고 도배 인원은 몇명이 필요해',
 '롱브릭타일의 단점이 무엇인지, 그리고 줄퍼티 마감이 뭐야?',
 '폴리싱 타일의 단점이 무엇인지, 그리고 스마트홈 시스템을 집에 설치하려고 하는데, 보안에 대해 어떤 조치를 취해야 해?',
 '샤시가 무엇이며, 다운라이트에 대해 설명해줘',
 '외벽 모서리 부위에 결로가 발생하는 원인이 무엇이며, 친환경적인 인테리어 디자인을 추구하려면 어떤 재료와 소품을 사용해야 하나요?',
 '금속기와의 단점은 무엇이고, 강화마루의 단점을 설명해줘',
 '협소한 공간에서도 쾌적한 인테리어를 구성하는 방법은 무엇인지, 또한, 도배지에 반점이 생겼을 때 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 방법의 장점이 뭐야?',
 '올퍼티 마감이 무엇이고, 합판마루는 보통 얼마정도 해?',
 '모던한 주방 디자인을 위한 인테리어 아이디어가 무엇이고, 어떻게 낮은 천장을 가진 공간을 더 높아 보이게 만들 수 있을까요?',
 '청고벽돌의 단점이 무엇이고, 다운라이트에 대해 설명해줘',
 '리얼징크의 장점은 무엇인지, 또한, 비닐사이딩의 단점이 뭐야?',
 '데코타일이 무엇이고, 단열벽지에 대해 알려줘',
 '다공질 흡음재 중 타공보드가 무엇이고, 어떤 환경조건으로 인해 몰딩수정이 발생할 수 있어?',
 '도막이 무엇인지, 또 세라믹 타일의 단점이 뭐야?',
 '프리케스트콘크리트 구조가 무엇인지, 그리고 도배지 꼬임이 발생하는 주요 원인을 알려줘',
 '페인트 상도재가 무엇이며, 층간소음

In [118]:
train_ft['qna_category'] = [[]] * len(train_ft)
train_ft

,category,question,answer,qna,qna_category
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...,[]
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.",[]
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...,[]
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사...",[]
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.,[]
...,...,...,...,...,...
132215,"[건축구조, 마감재]","경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ...",[]
132216,[마감재],"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나...",[]
132217,[마감재],"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘",모노롱브릭타일은 일반 시멘트벽돌과는 다르게 얇고 긴 모양의 타일 형태로 와이드 벽돌...,"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘 모노롱브릭타일은 일반 시멘트벽돌과...",[]
132218,[마감재],"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로...",[]


In [119]:
train_ft

,category,question,answer,qna,qna_category
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...,[]
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.",[]
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...,[]
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사...",[]
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.,[]
...,...,...,...,...,...
132215,"[건축구조, 마감재]","경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ...",[]
132216,[마감재],"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나...",[]
132217,[마감재],"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘",모노롱브릭타일은 일반 시멘트벽돌과는 다르게 얇고 긴 모양의 타일 형태로 와이드 벽돌...,"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘 모노롱브릭타일은 일반 시멘트벽돌과...",[]
132218,[마감재],"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로...",[]


### 장점

In [120]:
mask = train_ft['question'].str.contains('장점|좋은점|좋은 점|이점|장단점') & ~train_ft['question'].str.contains('차이점')

train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['장점'])

train_ft[mask]
# train_ft.loc[mask, 'qna_category'] = '장단점'

,category,question,answer,qna,qna_category
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...,[장점]
65,[마감재],오리지널징크의 장점이 뭐야?,오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경...,오리지널징크의 장점이 뭐야? 오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패...,[장점]
74,[마감재],그라스울의 장점이 뭐야?,"그라스울은 단열성능이 뛰어나고 운반, 보관 및 취급이 용이하다는 장점이 있습니다.","그라스울의 장점이 뭐야? 그라스울은 단열성능이 뛰어나고 운반, 보관 및 취급이 용이...",[장점]
78,[마감재],데코타일의 장점이 뭐야?,"데코타일은 가격이 저렴하고 시공이 간단하며 내구성, 내수성, 내마모성이 우수합니다.","데코타일의 장점이 뭐야? 데코타일은 가격이 저렴하고 시공이 간단하며 내구성, 내수성...",[장점]
92,[마감재],원목마루의 장점이 뭐야?,원목마루는 원목 그대로의 자연스러움을 연출하고 친환경적이며 보행감과 촉감이 좋고 충...,원목마루의 장점이 뭐야? 원목마루는 원목 그대로의 자연스러움을 연출하고 친환경적이며...,[장점]
...,...,...,...,...,...
132202,"[마감재, 마감하자]","방부목의 장점이 무엇인지, 또한, 내외부 온도의 큰 차이에 의해 곰팡이가 발생할 수...","방부목은 데크재 중 가장 저렴하다는 장점입니다. 또한, 내외부 온도차가 클 경우 도...","방부목의 장점이 무엇인지, 또한, 내외부 온도의 큰 차이에 의해 곰팡이가 발생할 수...",[장점]
132204,[마감재],"탄화목사이딩의 장점이 무엇인지, 또 수성페인트의 특징이 뭐야?",탄화목사이딩은 일반 목재보다 내구성이 좋고 관리가 쉬우며 친환경적인 점이 장점입니다...,"탄화목사이딩의 장점이 무엇인지, 또 수성페인트의 특징이 뭐야? 탄화목사이딩은 일반 ...",[장점]
132205,"[시공, 마감재]","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 흡음재 중 다공질은 뭐...","내단열 시공은 외단열 시공에 비해 시공 난이도가 낮으며 상대적으로 공사비, 재료비,...","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 흡음재 중 다공질은 뭐...",[장점]
132208,[마감재],"롱브릭타일의 장점이 무엇이며, 도막이 뭐야?",롱브릭타일은 점토벽돌보다 강도가 높고 시공비 측면에서 경제적이란 점이 장점입니다. ...,"롱브릭타일의 장점이 무엇이며, 도막이 뭐야? 롱브릭타일은 점토벽돌보다 강도가 높고 ...",[장점]


### 단점

In [121]:
mask = train_ft['question'].str.contains('단점|어디가 안좋아|어떤 점이 안좋|어떤 점이 안 좋|부작용|장단점')

train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['단점'])
train_ft[mask]


,category,question,answer,qna,qna_category
10,[건축구조],철골구조의 단점이 뭐야?,철골구조는 화재 발생시 철골이 녹아 건물의 붕괴로 이어질 수 있다는 단점이 있습니다.,철골구조의 단점이 뭐야? 철골구조는 화재 발생시 철골이 녹아 건물의 붕괴로 이어질 ...,[단점]
31,[기타],아세톤에 노출되면 어디가 안좋아?,"아세톤은 페인트와 같은 물질에 있는 독성물질로, 이에 노출되면 언어장애, 두통, 협...","아세톤에 노출되면 어디가 안좋아? 아세톤은 페인트와 같은 물질에 있는 독성물질로, ...",[단점]
58,[마감재],원목마루의 단점이 뭐야?,원목마루는 마루 중 가장 비싼 편이고 원목이 두께감이 있어 열전도율이 낮으며 습기에...,원목마루의 단점이 뭐야? 원목마루는 마루 중 가장 비싼 편이고 원목이 두께감이 있어...,[단점]
61,[마감재],평지붕의 단점은 뭐야?,"평지붕의 단점은 경사지붕보다 에너지 손실이 크고 단열, 방수를 위한 시공비가 많이들...","평지붕의 단점은 뭐야? 평지붕의 단점은 경사지붕보다 에너지 손실이 크고 단열, 방수...",[단점]
67,[마감재],합지벽지의 단점이 뭐야?,합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하며 오염이 생기면 닦거나 청소...,합지벽지의 단점이 뭐야? 합지벽지의 단점은 상대적으로 수명이 짧고 내구성이 약하며 ...,[단점]
...,...,...,...,...,...
132201,[마감재],"수성페인트의 특징이 무엇이고, 파벽돌의 단점이 뭐야?","수성페인트는 건물외벽이나 콘크리트, 시멘트 벽, 목재, 벽지 등에 주로 사용됩니다....","수성페인트의 특징이 무엇이고, 파벽돌의 단점이 뭐야? 수성페인트는 건물외벽이나 콘크...",[단점]
132209,"[건축구조, 마감재]","커튼월이 무엇인지, 또 스타코 플렉스의 단점은 뭐야?","커튼월은 건물의 하중을 모두 기중, 들보, 바닥, 지붕으로 지탱하고, 외벽은 하중을...","커튼월이 무엇인지, 또 스타코 플렉스의 단점은 뭐야? 커튼월은 건물의 하중을 모두 ...",[단점]
132214,"[마감재, 마감하자]","징크판넬의 단점이 무엇이며, 높은 습도로 인해 도배지와 몰딩면에 틈새가 생길 수 있어?",징크판넬의 단점은 다른 징크에 비해 수명이 짧은 편이고 시공이 잘못될 경우 녹슬기 ...,"징크판넬의 단점이 무엇이며, 높은 습도로 인해 도배지와 몰딩면에 틈새가 생길 수 있...",[단점]
132216,[마감재],"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나...",[단점]


### 종류

In [122]:
mask = train_ft['question'].str.contains('종류|어떤 방식|포함|다양한 방식|뭐가 있어|무엇이 있어?')

train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['종류'])
train_ft[mask]

,category,question,answer,qna,qna_category
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.",[종류]
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.,[종류]
11,[건축구조],콘크리트 구조는 어떤 방식이 있어?,"콘크리트는 철근콘크리트구조, 철골콘크리트구조, 철골철근콘크리트구조, 프리케스트콘크리...","콘크리트 구조는 어떤 방식이 있어? 콘크리트는 철근콘크리트구조, 철골콘크리트구조, ...",[종류]
18,[건축구조],조적식 구조는 어떤 방식이 있어?,"조적식 구조는 벽돌구조, 블록구조, 석구조가 있습니다.","조적식 구조는 어떤 방식이 있어? 조적식 구조는 벽돌구조, 블록구조, 석구조가 있습니다.",[종류]
19,[건축구조],목구조는 어떤 방식이 있어?,"목구조는 통나무구조 방식, 중목구조 방식, 기둥-보 구조방식, 경량목구조 방식이 있...","목구조는 어떤 방식이 있어? 목구조는 통나무구조 방식, 중목구조 방식, 기둥-보 구...",[종류]
...,...,...,...,...,...
132162,[마감재],"석고보드를 사용하는 이유가 무엇인지, 또한, 차음재 종류는 뭐가 있어?",석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,"석고보드를 사용하는 이유가 무엇인지, 또한, 차음재 종류는 뭐가 있어? 석고보드는 ...",[종류]
132184,"[건축구조, 마감재]","제진구조가 무엇인지, 그리고 화재 관련 마감재 종류가 뭐가 있어?","제진구조란 지진이 발생 시 관성에 의하여 진동을 일으키는 지진 에너지가 발생되는데,...","제진구조가 무엇인지, 그리고 화재 관련 마감재 종류가 뭐가 있어? 제진구조란 지진이...",[종류]
132185,"[마감하자, 시공]","도배지 꼬임의 해결방법이 무엇인지, 도배 부자재에는 어떤 것이 포함되어 있어?",도배지 꼬임 현상은 발생한 원인에 따라 적절한 조치를 취하여 해결해야 합니다. 1...,"도배지 꼬임의 해결방법이 무엇인지, 도배 부자재에는 어떤 것이 포함되어 있어? 도...",[종류]
132205,"[시공, 마감재]","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 흡음재 중 다공질은 뭐...","내단열 시공은 외단열 시공에 비해 시공 난이도가 낮으며 상대적으로 공사비, 재료비,...","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 흡음재 중 다공질은 뭐...","[장점, 종류]"


### 기능.효과

In [123]:
mask = train_ft['question'].str.contains('기능|효과(?!적|있는)')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['기능.효과'])

train_ft[mask]

,category,question,answer,qna,qna_category
192,[마감재],방청 페인트의 기능이 뭐야?,"방청 페인트의 기능은 철재 부식 방지, 철과 페인트의 접착력 증대, 철근의 방청도료...","방청 페인트의 기능이 뭐야? 방청 페인트의 기능은 철재 부식 방지, 철과 페인트의 ...",[기능.효과]
279,[마감재],몰딩의 기능이 뭐야?,"몰딩은 바닥과 벽, 벽과 벽 등의 접합부분을 깔끔하게 정리해주는 역할을 합니다.","몰딩의 기능이 뭐야? 몰딩은 바닥과 벽, 벽과 벽 등의 접합부분을 깔끔하게 정리해주...",[기능.효과]
549,[인테리어],아레카 야자의 효과는 뭐야?,"아레카 야자는 NASA에서 선정한 공기정화식물 1위에 해당하는 식물로 포름알데히드,...",아레카 야자의 효과는 뭐야? 아레카 야자는 NASA에서 선정한 공기정화식물 1위에 ...,[기능.효과]
590,[인테리어],스파티필름의 효과는 뭐야?,"연세대학교 의과대학교 분석실 실험에 따르면 스파티필름의 효과는 벤젠, 톨루엔, 에딜...",스파티필름의 효과는 뭐야? 연세대학교 의과대학교 분석실 실험에 따르면 스파티필름의 ...,[기능.효과]
597,[인테리어],거미식물의 효과는 뭐야?,"거미식물의 효과는 공기 중 독성물질(일산화탄소, 포름알데히드, 자일렌, 톨루엔 등)...","거미식물의 효과는 뭐야? 거미식물의 효과는 공기 중 독성물질(일산화탄소, 포름알데히...",[기능.효과]
...,...,...,...,...,...
132004,"[인테리어, 마감재]","거미식물의 효과는 무엇인지, 또한, 쌀풀에 대해 알려줘","거미식물의 효과는 공기 중 독성물질(일산화탄소, 포름알데히드, 자일렌, 톨루엔 등)...","거미식물의 효과는 무엇인지, 또한, 쌀풀에 대해 알려줘 거미식물의 효과는 공기 중 ...",[기능.효과]
132045,"[인테리어, 마감재]","아레카 야자의 효과는 무엇이며, 우레탄 계열 페인트는 어디에 사용해?","아레카 야자는 NASA에서 선정한 공기정화식물 1위에 해당하는 식물로 포름알데히드,...","아레카 야자의 효과는 무엇이며, 우레탄 계열 페인트는 어디에 사용해? 아레카 야자는...",[기능.효과]
132118,"[마감재, 인테리어]","데코타일의 장점이 무엇이고, 아레카 야자의 효과는 뭐야?","데코타일은 가격이 저렴하고 시공이 간단하며 내구성, 내수성, 내마모성이 우수합니다....","데코타일의 장점이 무엇이고, 아레카 야자의 효과는 뭐야? 데코타일은 가격이 저렴하고...","[장점, 기능.효과]"
132129,"[인테리어, 건축구조]","아레카 야자의 효과는 무엇이며, 통나무구조 방식이 뭐야?","아레카 야자는 NASA에서 선정한 공기정화식물 1위에 해당하는 식물로 포름알데히드,...","아레카 야자의 효과는 무엇이며, 통나무구조 방식이 뭐야? 아레카 야자는 NASA에서...",[기능.효과]


### 용도

In [124]:
mask = train_ft['question'].str.contains('용도|어디에 사용|사용하는 이유|위해 사용')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['목적.용도'])

train_ft[mask]

,category,question,answer,qna,qna_category
85,[마감재],우레탄 계열 페인트는 어디에 사용해?,우레탄 계열 페인트는 탄성이 좋아 보통 방수기능이 필요한 곳에 적용합니다.,우레탄 계열 페인트는 어디에 사용해? 우레탄 계열 페인트는 탄성이 좋아 보통 방수기...,[목적.용도]
89,[마감재],방청 페인트는 어디에 사용해?,"방청 페인트는 철골 구조물, 자동차 및 가전제품 철재 부속품, 선박 및 컨테이너박스...","방청 페인트는 어디에 사용해? 방청 페인트는 철골 구조물, 자동차 및 가전제품 철재...",[목적.용도]
126,[마감재],에폭시 계열 페인트는 어디에 사용해?,에폭시 계열 페인트는 강도가 좋아 보도 및 바닥재 등 높은 강도를 요구하는 곳에 적...,에폭시 계열 페인트는 어디에 사용해? 에폭시 계열 페인트는 강도가 좋아 보도 및 바...,[목적.용도]
169,[마감재],석고보드를 사용하는 이유가 뭐야?,석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,석고보드를 사용하는 이유가 뭐야? 석고보드는 인체에 무해하며 친환경적이므로 마감재로...,[목적.용도]
207,[마감재],에나멜 계열 페인트는 어디에 사용해?,에나멜 게열 페인트는 유기염료를 이용하여 색상을 내기 편하고 내화학성이 높아 플라스...,에나멜 계열 페인트는 어디에 사용해? 에나멜 게열 페인트는 유기염료를 이용하여 색상...,[목적.용도]
...,...,...,...,...,...
131904,[마감재],"뮤럴벽지가 무엇이며, 아크릴 계열 페인트는 어디에 사용해?","그림, 벽화라는 의미의 뮤럴벽지는 맞춤벽지라 별도의 제작기간이 필요하다는 특징이 있...","뮤럴벽지가 무엇이며, 아크릴 계열 페인트는 어디에 사용해? 그림, 벽화라는 의미의 ...",[목적.용도]
131952,[마감재],"석고보드를 사용하는 이유가 무엇인지, 또 페인트가 남으면 어떻게 처리해?",석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,"석고보드를 사용하는 이유가 무엇인지, 또 페인트가 남으면 어떻게 처리해? 석고보드는...",[목적.용도]
132045,"[인테리어, 마감재]","아레카 야자의 효과는 무엇이며, 우레탄 계열 페인트는 어디에 사용해?","아레카 야자는 NASA에서 선정한 공기정화식물 1위에 해당하는 식물로 포름알데히드,...","아레카 야자의 효과는 무엇이며, 우레탄 계열 페인트는 어디에 사용해? 아레카 야자는...","[기능.효과, 목적.용도]"
132104,"[마감재, 시공]","석고보드를 사용하는 이유가 무엇인지, 또한, 초배작업이 뭐야?",석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,"석고보드를 사용하는 이유가 무엇인지, 또한, 초배작업이 뭐야? 석고보드는 인체에 무...",[목적.용도]


### 차이

In [125]:
mask = train_ft['question'].str.contains('관계|차이점|차이(?!에)')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['차이'])

train_ft[mask]

,category,question,answer,qna,qna_category
15,[건축구조],면진구조와 제진구조의 차이점이 뭐야?,면진구조는 지진력이 구조물에 상대적으로 약하게 전달되도록 수평방향으로 유연한 장치를...,면진구조와 제진구조의 차이점이 뭐야? 면진구조는 지진력이 구조물에 상대적으로 약하게...,[차이]
35,[기타],방수와 발수의 차이가 뭐야?,"방수는 제품 표면에 파라핀, 플라스틱이 우레탄 같은 코팅막을 형성해서 제품에 소수성...","방수와 발수의 차이가 뭐야? 방수는 제품 표면에 파라핀, 플라스틱이 우레탄 같은 코...",[차이]
352,[마감하자],훼손과 오염의 차이는 뭐야?,"훼손과 오염의 차이는 물리적인 변화의 유무입니다. 훼손은 찍힘, 터짐 등의 물리적인...",훼손과 오염의 차이는 뭐야? 훼손과 오염의 차이는 물리적인 변화의 유무입니다. 훼손...,[차이]
776,[건축구조],면진구조와 제진구조의 차이점이 뭐야?,면진구조는 지진력이 건물에 상대적으로 약하게 전달되도록 수평방향으로 유연한 장치를 ...,면진구조와 제진구조의 차이점이 뭐야? 면진구조는 지진력이 건물에 상대적으로 약하게 ...,[차이]
796,[기타],방수와 발수의 차이가 뭐야?,"방수는 제품 표면에 파라핀, 플라스틱 또는 우레탄과 같은 코팅막을 형성하여 제품에 ...","방수와 발수의 차이가 뭐야? 방수는 제품 표면에 파라핀, 플라스틱 또는 우레탄과 같...",[차이]
...,...,...,...,...,...
132111,[마감하자],"훼손과 오염의 차이는 무엇인지, 그리고 피스 하자가 뭐야?","훼손과 오염의 차이는 물리적인 변화의 유무입니다. 훼손은 찍힘, 터짐 등의 물리적인...","훼손과 오염의 차이는 무엇인지, 그리고 피스 하자가 뭐야? 훼손과 오염의 차이는 물...",[차이]
132114,"[기타, 마감재]","방수와 발수의 차이가 무엇인지, 또한, 페인트가 뭐야?","방수는 제품 표면에 파라핀, 플라스틱이 우레탄 같은 코팅막을 형성해서 제품에 소수성...","방수와 발수의 차이가 무엇인지, 또한, 페인트가 뭐야? 방수는 제품 표면에 파라핀,...",[차이]
132131,"[마감하자, 건축구조]","훼손과 오염의 차이는 무엇인지, 또 제진구조가 뭐야?","훼손과 오염의 차이는 물리적인 변화의 유무입니다. 훼손은 찍힘, 터짐 등의 물리적인...","훼손과 오염의 차이는 무엇인지, 또 제진구조가 뭐야? 훼손과 오염의 차이는 물리적인...",[차이]
132160,"[마감재, 마감하자]","시멘트사이딩의 장점이 무엇이고, 훼손과 오염의 차이는 뭐야?",시멘트사이딩은 다양한 색상 연출이 가능하고 다른 자재에 비래 저렴하며 시공 및 유지...,"시멘트사이딩의 장점이 무엇이고, 훼손과 오염의 차이는 뭐야? 시멘트사이딩은 다양한 ...","[장점, 차이]"


### 비교

In [126]:
mask = train_ft['question'].str.contains('뭐가 더|비교|비해|더 좋은|중 어느 것')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['비교'])

train_ft[mask]

,category,question,answer,qna,qna_category
154,[마감재],도배와 페인트 시공 중 뭐가 더 비용이 많이 들어?,일반적으로 도배보다 페인트 시공이 더 많은 비용이 듭니다. 페인트는 특성상 2~3번...,도배와 페인트 시공 중 뭐가 더 비용이 많이 들어? 일반적으로 도배보다 페인트 시공...,[비교]
166,[마감재],아이가 있는 집은 두꺼운 장판과 마루 중 어느 것이 좋아?,아이가 있는 집에는 두꺼운 장판을 추천드립니다. 아이가 있는 경우 층간 소음이 발생...,아이가 있는 집은 두꺼운 장판과 마루 중 어느 것이 좋아? 아이가 있는 집에는 두꺼...,[비교]
410,[시공],왜 실크벽지가 합지 벽지에 비해 필요한 도배사가 더 많아?,실크벽지의 경우 합지벽지와 달리 초배작업이 필요하므로 도배사가 필요합니다.,왜 실크벽지가 합지 벽지에 비해 필요한 도배사가 더 많아? 실크벽지의 경우 합지벽지...,[비교]
424,[시공],외단열 시공이 내단열 시공에 비해 갖는 장점이 뭐야?,"외단열 시공은 내단열 시공에 비해 단열효과가 좋으며, 면적확보가 용이합니다.",외단열 시공이 내단열 시공에 비해 갖는 장점이 뭐야? 외단열 시공은 내단열 시공에 ...,"[장점, 비교]"
477,[시공],내단열 시공에 비해 외단열 시공이 갖는 단점이 뭐야?,내단열 시공에 비해 외단열 시공은 상대적으로 시공 난이도가 높아 전문 시공자가 필요...,내단열 시공에 비해 외단열 시공이 갖는 단점이 뭐야? 내단열 시공에 비해 외단열 시...,"[단점, 비교]"
...,...,...,...,...,...
131917,"[시공, 인테리어]","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 인테리어에서 휴식과 안...","내단열 시공은 외단열 시공에 비해 시공 난이도가 낮으며 상대적으로 공사비, 재료비,...","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 인테리어에서 휴식과 안...","[장점, 비교]"
131949,"[시공, 인테리어]","내단열 시공에 비해 외단열 시공이 갖는 단점이 무엇이고, 어떻게 유럽의 클래식한 디...",내단열 시공에 비해 외단열 시공은 상대적으로 시공 난이도가 높아 전문 시공자가 필요...,"내단열 시공에 비해 외단열 시공이 갖는 단점이 무엇이고, 어떻게 유럽의 클래식한 디...","[단점, 비교]"
131976,[시공],"겨울철 도배를 할 때 주의해야 할 점이 무엇인지, 또 외단열 시공에 비해 내단열 시...","도배 시공 직후 집안의 냄새를 제거하려는 용도로 환기를 하는 것은 좋지만, 춥고 건...","겨울철 도배를 할 때 주의해야 할 점이 무엇인지, 또 외단열 시공에 비해 내단열 시...","[단점, 비교]"
132094,"[시공, 마감재]","내단열 시공에 비해 외단열 시공이 갖는 단점이 무엇이고, 전벽돌이 뭐야?",내단열 시공에 비해 외단열 시공은 상대적으로 시공 난이도가 높아 전문 시공자가 필요...,"내단열 시공에 비해 외단열 시공이 갖는 단점이 무엇이고, 전벽돌이 뭐야? 내단열 시...","[단점, 비교]"


### 원인

In [127]:
mask = train_ft['question'].str.contains('원인|문제의 원인')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['원인'])

train_ft[mask]

,category,question,answer,qna,qna_category
49,[기타],새집증후군의 원인이 뭐야?,새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이상 증상이 신체에 나타나는 ...,새집증후군의 원인이 뭐야? 새집증후군은 새솔 지은 집에 입주했을 때 이전에 없던 이...,[원인]
297,[마감재],벽지가 변색이 되는 원인이 뭐가 있어?,"벽지가 변색이 되는 이유는 습기, 외부 오염, 직사광선 노출 등이 있습니다.","벽지가 변색이 되는 원인이 뭐가 있어? 벽지가 변색이 되는 이유는 습기, 외부 오염...","[종류, 원인]"
373,[마감하자],벽지에 반점이 생기는 원인이 뭐야?,대한건축학회와 내화건축자재협회에서 공동으로 진행한 연구결과에 따르면 벽지에 반점이 ...,벽지에 반점이 생기는 원인이 뭐야? 대한건축학회와 내화건축자재협회에서 공동으로 진행...,[원인]
625,[타마감하자],결로가 발생하는 원인이 뭐야?,결로가 발생하는 원인은 높은 습도와 차가운 표면온도가 있습니다.,결로가 발생하는 원인이 뭐야? 결로가 발생하는 원인은 높은 습도와 차가운 표면온도가...,[원인]
629,[타마감하자],발코니에 결로가 발생하는 원인은 뭐야?,발코니에 결로가 발생하는 원인은 발코니에 단열 및 방습조치가 되지 않은 상태에서 실...,발코니에 결로가 발생하는 원인은 뭐야? 발코니에 결로가 발생하는 원인은 발코니에 단...,[원인]
...,...,...,...,...,...
132159,"[마감하자, 인테리어]","도배지에 녹이 발생하는 원인이 무엇이며, 스파티필름의 효과는 뭐야?",높은 습도로 인해 도배지 안쪽의 금속 물질이 산화되어 녹이 도배지에 발생할 수 있습...,"도배지에 녹이 발생하는 원인이 무엇이며, 스파티필름의 효과는 뭐야? 높은 습도로 인...","[기능.효과, 원인]"
132164,"[타마감하자, 시공]","AD, PD에 면한 벽체 결로의 원인이 무엇이고, 왜 초배지만 남은 벽에는 페인트를...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인이 무엇이고, 왜 초배지만 남은 벽에는 페인트를...",[원인]
132177,"[마감재, 마감하자]","오리지널징크가 무엇인지, 그리고 도배지에 곰팡이가 생기는 원인 알려줘",오리지널 징크는 순도 99.995%의 아연과 소량의 티타늄과 구리가 첨가된 합금 판...,"오리지널징크가 무엇인지, 그리고 도배지에 곰팡이가 생기는 원인 알려줘 오리지널 징크...",[원인]
132194,"[타마감하자, 마감재]","지하 공간 결로의 원인이 무엇인지, 또한, 강화마루를 사용할 때 주의해야 하는 점 알려줘",지하 공간에서 발생하는 결로의 원인은 습한 공기입니다. 지하 공간은 일사에 의한 영...,"지하 공간 결로의 원인이 무엇인지, 또한, 강화마루를 사용할 때 주의해야 하는 점 ...",[원인]


### 주의점

In [128]:
mask = train_ft['question'].str.contains('(?!부|자연)주의|유의|주의사항|특별히|조심')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['주의점'])

train_ft[mask]

,category,question,answer,qna,qna_category
48,[기타],분말 소화기를 사용할 때 주의할 점이 뭐야?,"분말 소화기를 사용할 때 주의할 점은 압력계의 눈금 위치가 초록색에 있는지 확인, ...",분말 소화기를 사용할 때 주의할 점이 뭐야? 분말 소화기를 사용할 때 주의할 점은 ...,[주의점]
347,[마감하자],부주의한 시공으로 인해 도배지 패턴이 이어지지 않을 수 있어?,시공자의 부주의에 의해 도배지 무늬 패턴에 맞지 않을 수 있습니다.,부주의한 시공으로 인해 도배지 패턴이 이어지지 않을 수 있어? 시공자의 부주의에 의...,[주의점]
400,[시공],배관공사할 때 주의해야 할 점이 뭐야?,"배관공사 할 때 주의할 점은 기울기, 통기구 설치, 소제구 설치입니다.","배관공사할 때 주의해야 할 점이 뭐야? 배관공사 할 때 주의할 점은 기울기, 통기구...",[주의점]
428,[시공],겨울에 도배 주의사항 좀 알려줘,겨울철 도배하기 위한 주의사항을 알려드리겠습니다. 1. 일정한 온도 유지 추운 겨울...,겨울에 도배 주의사항 좀 알려줘 겨울철 도배하기 위한 주의사항을 알려드리겠습니다. ...,[주의점]
438,[시공],장롱 옮길 때 주의할 점이 뭐야?,장롱을 옮길 때 주의할 점은 장판이 상하지 않도록 헝겁 등의 천을 대는 것이 좋습니...,장롱 옮길 때 주의할 점이 뭐야? 장롱을 옮길 때 주의할 점은 장판이 상하지 않도록...,[주의점]
...,...,...,...,...,...
132176,"[시공, 마감하자]","장롱 옮길 때 주의할 점이 무엇이고, 도배지에 반점이 생겼어",장롱을 옮길 때 주의할 점은 장판이 상하지 않도록 헝겁 등의 천을 대는 것이 좋습니...,"장롱 옮길 때 주의할 점이 무엇이고, 도배지에 반점이 생겼어 장롱을 옮길 때 주의할...",[주의점]
132194,"[타마감하자, 마감재]","지하 공간 결로의 원인이 무엇인지, 또한, 강화마루를 사용할 때 주의해야 하는 점 알려줘",지하 공간에서 발생하는 결로의 원인은 습한 공기입니다. 지하 공간은 일사에 의한 영...,"지하 공간 결로의 원인이 무엇인지, 또한, 강화마루를 사용할 때 주의해야 하는 점 ...","[원인, 주의점]"
132198,"[마감재, 시공]","합지벽지가 무엇이며, 겨울에 도배 주의사항 좀 알려줘","합지벽지란 종이를 2장 이상 겹쳐서 만든 벽지를 의미합니다. 가격은 평당 3,000...","합지벽지가 무엇이며, 겨울에 도배 주의사항 좀 알려줘 합지벽지란 종이를 2장 이상 ...",[주의점]
132212,"[기타, 마감재]","분말 소화기를 사용할 때 주의할 점이 무엇인지, 또한, 천연페인트의 특징이 뭐야?","분말 소화기를 사용할 때 주의할 점은 압력계의 눈금 위치가 초록색에 있는지 확인, ...","분말 소화기를 사용할 때 주의할 점이 무엇인지, 또한, 천연페인트의 특징이 뭐야? ...",[주의점]


### 방법

In [129]:
mask = train_ft['question'].str.contains('방법|법|어떻게|방법은') & ~train_ft['question'].str.contains('압출법|해결 방법|해결|어떻게 돼|대책')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['방법'])

train_ft[mask]

,category,question,answer,qna,qna_category
17,[건축구조],건축물의 구조는 어떻게 구분해?,"건축물의 구조는 목구조, 조적조, 철골구조, 콘크리트구조로 구별할 수 있습니다.","건축물의 구조는 어떻게 구분해? 건축물의 구조는 목구조, 조적조, 철골구조, 콘크리...",[방법]
22,[건축구조],화재에 취약한 철골구조의 보완법에 뭐가 있을까?,"철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다...",화재에 취약한 철골구조의 보완법에 뭐가 있을까? 철골구조는 화재 발생 시 내구성이 ...,[방법]
34,[기타],아파트 도배평수는 어떻게 게산해?,아파트 도배평수는 일반적으로 분양 평수*2.5로 계산합니다.,아파트 도배평수는 어떻게 게산해? 아파트 도배평수는 일반적으로 분양 평수*2.5로 ...,[방법]
44,[기타],새집증후군을 예방하는 방법이 뭐야?,새집증후군을 예방하는 방법은 실내흡연을 하지않고 포르말린을 사용하지 않은 목재 가구...,새집증후군을 예방하는 방법이 뭐야? 새집증후군을 예방하는 방법은 실내흡연을 하지않고...,[방법]
47,[기타],마감재의 하자여부는 어떻게 판단해?,"마감재의 하자 여부는 설계도서와의 일치 여부, 기능상의 문제 여부, 미관상의 문제 ...","마감재의 하자여부는 어떻게 판단해? 마감재의 하자 여부는 설계도서와의 일치 여부, ...",[방법]
...,...,...,...,...,...
132200,"[마감하자, 인테리어]",도배지에 반점이 생겼을 때 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 ...,상대적으로 간단한 해결책으로 바로 수정이 가능합니다. 또 친환경적인 인테리어를 위해...,도배지에 반점이 생겼을 때 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 ...,"[장점, 방법]"
132207,"[마감재, 시공]","알루미늄징크가 무엇이고, 피도면 정리는 어떻게 작업해?",알루미늄징크는 은백색의 가벼운 알루미늄 판에 아연 도금을 하여 색상페인팅을 한 것 ...,"알루미늄징크가 무엇이고, 피도면 정리는 어떻게 작업해? 알루미늄징크는 은백색의 가벼...",[방법]
132215,"[건축구조, 마감재]","경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ...","[장점, 방법]"
132218,[마감재],"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로...","[단점, 방법]"


### 대책

In [130]:
mask = train_ft['question'].str.contains('해결|문제 해결|해결 방법|대책|조치|예방')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['대책'])

train_ft[mask]

,category,question,answer,qna,qna_category
44,[기타],새집증후군을 예방하는 방법이 뭐야?,새집증후군을 예방하는 방법은 실내흡연을 하지않고 포르말린을 사용하지 않은 목재 가구...,새집증후군을 예방하는 방법이 뭐야? 새집증후군을 예방하는 방법은 실내흡연을 하지않고...,"[방법, 대책]"
51,[기타],"스마트홈 시스템을 집에 설치하려고 하는데, 보안에 대해 어떤 조치를 취해야 해?","스마트홈 시스템을 설치할 때 보안은 매우 중요한 요소입니다. 첫째, 강력한 비밀번호...","스마트홈 시스템을 집에 설치하려고 하는데, 보안에 대해 어떤 조치를 취해야 해? 스...",[대책]
55,[기타],새집증후군의 해결 방법이 뭐야?,"새집증후군을 해결하려면 천연 소재, 포름알데히드 처리를 하지 않은 자재를 사용해야하...","새집증후군의 해결 방법이 뭐야? 새집증후군을 해결하려면 천연 소재, 포름알데히드 ...",[대책]
443,[시공],도어 프레임이 변형되어 문이 제대로 닫히지 않을 때 어떻게 해결하나요?,프레임을 조정하여 수평으로 맞추고 문을 다시 설치하거나 교체하세요.,도어 프레임이 변형되어 문이 제대로 닫히지 않을 때 어떻게 해결하나요? 프레임을 조...,[대책]
445,[시공],내부 문이 제대로 열리지 않거나 닫히지 않을 때 어떻게 조치하나요?,"문틀이 치우친 경우, 맞추거나 조정하여 문이 제대로 열리고 닫히도록 하세요.",내부 문이 제대로 열리지 않거나 닫히지 않을 때 어떻게 조치하나요? 문틀이 치우친 ...,"[방법, 대책]"
...,...,...,...,...,...
132180,"[타마감하자, 인테리어]","지하 공간 결로의 대책은 무엇인지, 또 거실을 현대적으로 꾸밀 때 고려해야 할 사항...",지하 공간 결로에 대한 대책은 제습시설 설치 및 단열재 보강입니다. 결로를 방질하기...,"지하 공간 결로의 대책은 무엇인지, 또 거실을 현대적으로 꾸밀 때 고려해야 할 사항...",[대책]
132182,"[마감하자, 시공]","몰딩 사이에 이격이 생겼을 경우 해결방법이 무엇이고, 도배 견적은 어떻게 계산해?","제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업...","몰딩 사이에 이격이 생겼을 경우 해결방법이 무엇이고, 도배 견적은 어떻게 계산해? ...",[대책]
132185,"[마감하자, 시공]","도배지 꼬임의 해결방법이 무엇인지, 도배 부자재에는 어떤 것이 포함되어 있어?",도배지 꼬임 현상은 발생한 원인에 따라 적절한 조치를 취하여 해결해야 합니다. 1...,"도배지 꼬임의 해결방법이 무엇인지, 도배 부자재에는 어떤 것이 포함되어 있어? 도...","[종류, 대책]"
132188,"[타마감하자, 기타]","창호 결로의 대책은 무엇인지, 그리고 아파트 도배평수는 어떻게 게산해?",창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,"창호 결로의 대책은 무엇인지, 그리고 아파트 도배평수는 어떻게 게산해? 창호결로의 ...",[대책]


### 계산

In [131]:
mask = train_ft['question'].str.contains('계산|평수|측정|지점|얼마나 필요')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['계산'])

train_ft[mask]

,category,question,answer,qna,qna_category
32,[기타],공동주택의 실내 라돈의 측정지점은 어떻게 돼?,공동주택의 실내 라돈은 최저층에서 측정하며 공동주택의 총 세대수가 100세대일 때 ...,공동주택의 실내 라돈의 측정지점은 어떻게 돼? 공동주택의 실내 라돈은 최저층에서 측...,[계산]
34,[기타],아파트 도배평수는 어떻게 게산해?,아파트 도배평수는 일반적으로 분양 평수*2.5로 계산합니다.,아파트 도배평수는 어떻게 게산해? 아파트 도배평수는 일반적으로 분양 평수*2.5로 ...,"[방법, 계산]"
41,[기타],라돈을 제외한 공동주택의 실내공기질 측정지점이 어떻게 돼?,라돈을 제외한 공동주택의 실내공기질 측정시료는 실내공기질공정시험기준에 의하여 100...,라돈을 제외한 공동주택의 실내공기질 측정지점이 어떻게 돼? 라돈을 제외한 공동주택의...,[계산]
42,[기타],사무실의 실내공기 시료채취 및 측정지점이 어떻게 돼?,실내공기의 측정시료는 사무실 안에서 공기질이 가장 나쁠 것으로 예상되는 2곳 이상에...,사무실의 실내공기 시료채취 및 측정지점이 어떻게 돼? 실내공기의 측정시료는 사무실 ...,[계산]
56,[기타],주택 도배평수는 어떻게 계산해?,주택 도배평수는 일반적으로 주택평수*3.5로 계산합니다.,주택 도배평수는 어떻게 계산해? 주택 도배평수는 일반적으로 주택평수*3.5로 계산합니다.,"[방법, 계산]"
...,...,...,...,...,...
131941,"[마감재, 기타]","세라믹사이딩이 무엇인지, 그리고 주택 도배평수는 어떻게 계산해?","세라믹사이딩은 가벼운 모래와 천연펄프, 콘크리트 등을 혼합해서 만든 것을 의미합니다...","세라믹사이딩이 무엇인지, 그리고 주택 도배평수는 어떻게 계산해? 세라믹사이딩은 가벼...","[방법, 계산]"
132016,"[인테리어, 기타]","인테리어에서 사용되는 바닥재 중 목재 바닥의 장점은 무엇인지, 또 라돈을 제외한 공...","목재 바닥은 따뜻한 느낌을 주며, 자연스러운 질감과 색상으로 공간에 풍부한 표현을 ...","인테리어에서 사용되는 바닥재 중 목재 바닥의 장점은 무엇인지, 또 라돈을 제외한 공...","[장점, 계산]"
132099,"[마감재, 시공]","방부목이 무엇인지, 또한, 도배 견적은 어떻게 계산해?","방부목이란 천연목재에 화학방부재를 주입하여 내구성을 강화한 목재입니다. 또한, 일반...","방부목이 무엇인지, 또한, 도배 견적은 어떻게 계산해? 방부목이란 천연목재에 화학방...","[방법, 계산]"
132182,"[마감하자, 시공]","몰딩 사이에 이격이 생겼을 경우 해결방법이 무엇이고, 도배 견적은 어떻게 계산해?","제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업...","몰딩 사이에 이격이 생겼을 경우 해결방법이 무엇이고, 도배 견적은 어떻게 계산해? ...","[대책, 계산]"


### 기간

In [132]:
mask = train_ft['question'].str.contains('시간|기간|수명|얼마나|주기') & ~train_ft['question'].str.contains('얼마나 필요')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['기간'])

train_ft[mask]

,category,question,answer,qna,qna_category
60,[마감재],카페트는 기대수명이 얼마야?,카페트의 기대수명은 6년입니다.,카페트는 기대수명이 얼마야? 카페트의 기대수명은 6년입니다.,[기간]
66,[마감재],실크벽지의 교체주기는 얼마야?,실크벽지의 교체주기는 5~7년입니다.,실크벽지의 교체주기는 얼마야? 실크벽지의 교체주기는 5~7년입니다.,[기간]
77,[마감재],강화마루는 기대수명이 얼마야?,강화마루의 기대수명은 25년입니다.,강화마루는 기대수명이 얼마야? 강화마루의 기대수명은 25년입니다.,[기간]
132,[마감재],폴리싱 타일의 기대수명은 얼마야?,폴리싱 타일의 기대수명은 50년입니다.,폴리싱 타일의 기대수명은 얼마야? 폴리싱 타일의 기대수명은 50년입니다.,[기간]
150,[마감재],도기질 타일의 기대수명은 얼마야?,도기질 타일의 기대수명은 50년입니다.,도기질 타일의 기대수명은 얼마야? 도기질 타일의 기대수명은 50년입니다.,[기간]
...,...,...,...,...,...
132101,[마감재],"금속기와의 장점은 무엇이고, 도기질 타일의 기대수명은 얼마야?","금속기와는 점토기와보다 가벼워 시공이 용이하고 저렴하며 지진, 태풍에 강하다는 것이...","금속기와의 장점은 무엇이고, 도기질 타일의 기대수명은 얼마야? 금속기와는 점토기와보...","[장점, 기간]"
132117,"[기타, 마감재]","분말 소화기를 사용할 때 주의할 점이 무엇인지, 또한, 에폭시 테라죠의 기대수명은 ...","분말 소화기를 사용할 때 주의할 점은 압력계의 눈금 위치가 초록색에 있는지 확인, ...","분말 소화기를 사용할 때 주의할 점이 무엇인지, 또한, 에폭시 테라죠의 기대수명은 ...","[주의점, 기간]"
132125,"[마감하자, 마감재]",도배지에 반점이 생겼을 때 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 ...,"재발 가능성이 있으며, 코팅 때문에 냄새가 발생할 수 있습니다. 또 대리석의 기대수...",도배지에 반점이 생겼을 때 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 ...,"[단점, 방법, 기간]"
132146,[마감재],"청고벽돌이 무엇인지, 또 대리석은 기대수명이 얼마야?","청고벽돌은 회색빛과 청색빛이 도는 고벽돌을 의미하며, 이전에 지어진 건물에 사용됐던...","청고벽돌이 무엇인지, 또 대리석은 기대수명이 얼마야? 청고벽돌은 회색빛과 청색빛이 ...",[기간]


### 시점

In [133]:
mask = train_ft['question'].str.contains('언제|시점|계절')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['시점'])

train_ft[mask]

,category,question,answer,qna,qna_category
411,[시공],도배하기 좋은 계절은 언제야?,도배하기 좋은 계절은 봄과 가을입니다. 봄과 가을은 계절적으로 선선하고 상대적으로 ...,도배하기 좋은 계절은 언제야? 도배하기 좋은 계절은 봄과 가을입니다. 봄과 가을은 ...,[시점]
420,[시공],도배할 때 추가비용은 언제 발생해?,"도배 시 시공환경, 발코니 확장 여부, 몰딩 및 문 교체 여부, 벽지 컬러 수 등에...","도배할 때 추가비용은 언제 발생해? 도배 시 시공환경, 발코니 확장 여부, 몰딩 및...",[시점]
461,[시공],배관공사에 소제구를 설치해야 하는 경우는 언제야?,배관공사에 소제구를 설치해야 하는 경우는 연결 부위가 직각일 때입니다. 배관의 연결...,배관공사에 소제구를 설치해야 하는 경우는 언제야? 배관공사에 소제구를 설치해야 하는...,[시점]
1172,[시공],도배하기 좋은 계절은 언제야?,"도배하기 좋은 계절은 봄과 가을입니다. 봄과 가을은 기온과 습도가 온화한 편이며, ...",도배하기 좋은 계절은 언제야? 도배하기 좋은 계절은 봄과 가을입니다. 봄과 가을은 ...,[시점]
1181,[시공],도배할 때 추가비용은 언제 발생해?,"도배 작업 시 추가 비용은 주로 시공환경, 발코니 확장 여부, 몰딩 및 문 교체 여...","도배할 때 추가비용은 언제 발생해? 도배 작업 시 추가 비용은 주로 시공환경, 발코...",[시점]
...,...,...,...,...,...
131375,"[기타, 시공]","층간소음의 기준이 무엇이고, 배관공사에 소제구를 설치해야 하는 경우는 언제야?","층간소음의 기준은 1분을 기준으로 주간에는 40dB, 야간에는 3dB을 넘을 경우에...","층간소음의 기준이 무엇이고, 배관공사에 소제구를 설치해야 하는 경우는 언제야? 층간...",[시점]
131464,"[마감재, 시공]","방염벽지의 단점이 무엇이고, 배관공사에 소제구를 설치해야 하는 경우는 언제야?","방염벽지는 합지, 실크벽지보다 비싸며 전문시공이 필요하여 시공비가 높아질 수 있다는...","방염벽지의 단점이 무엇이고, 배관공사에 소제구를 설치해야 하는 경우는 언제야? 방염...","[단점, 시점]"
131637,"[마감재, 시공]","써모사이딩의 단점이 무엇이고, 도배하기 좋은 계절은 언제야?",써모사이딩의 단점은 세라믹사이딩보다 퀄리티가 떨어지고 시간이 지나면 저렴한 자재의 ...,"써모사이딩의 단점이 무엇이고, 도배하기 좋은 계절은 언제야? 써모사이딩의 단점은 세...","[단점, 시점]"
131742,"[마감재, 시공]","비드법 보온판 1종의 장점이 무엇이며, 도배할 때 추가비용은 언제 발생해?",비드법 1종은 단열 성능이 좋고 시공 난이도가 낮으며 가격이 저렴하다는 장점이 있습...,"비드법 보온판 1종의 장점이 무엇이며, 도배할 때 추가비용은 언제 발생해? 비드법 ...","[장점, 방법, 시점]"


### 기준

In [134]:
mask = train_ft['question'].str.contains('판단|판정|기준|적절|적정|설정')   & ~train_ft['question'].str.contains('부적절')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['기준'])

train_ft[mask]

,category,question,answer,qna,qna_category
33,[기타],소화기 설치 기준이 뭐야?,소화기 설치 기준은 연면적 3.3제곱미터마다 설치해야 하고 각층마다 설치해야 하며 ...,소화기 설치 기준이 뭐야? 소화기 설치 기준은 연면적 3.3제곱미터마다 설치해야 하...,[기준]
38,[기타],층간소음의 기준이 뭐야?,"층간소음의 기준은 1분을 기준으로 주간에는 40dB, 야간에는 3dB을 넘을 경우에...","층간소음의 기준이 뭐야? 층간소음의 기준은 1분을 기준으로 주간에는 40dB, 야간...",[기준]
47,[기타],마감재의 하자여부는 어떻게 판단해?,"마감재의 하자 여부는 설계도서와의 일치 여부, 기능상의 문제 여부, 미관상의 문제 ...","마감재의 하자여부는 어떻게 판단해? 마감재의 하자 여부는 설계도서와의 일치 여부, ...","[방법, 기준]"
434,[시공],배관공사 시 기울기는 보통 얼마를 설정해?,배관공사 시 배관의 기울기는 원칙적으로 1/관경(배관직경)으로 설정합니다. 예를 들...,배관공사 시 기울기는 보통 얼마를 설정해? 배관공사 시 배관의 기울기는 원칙적으로 ...,[기준]
465,[시공],도배풀의 농도는 어느 정도가 적절해?,"적절한 도배풀의 농도는 도배풀 제조사, 생산연월, 도배지 종류, 작업시기 등에 따라...","도배풀의 농도는 어느 정도가 적절해? 적절한 도배풀의 농도는 도배풀 제조사, 생산연...",[기준]
...,...,...,...,...,...
131899,"[기타, 마감재]","소화기 설치 기준이 무엇이며, 흡음재 구조는 어떤 것이 있어?",소화기 설치 기준은 연면적 3.3제곱미터마다 설치해야 하고 각층마다 설치해야 하며 ...,"소화기 설치 기준이 무엇이며, 흡음재 구조는 어떤 것이 있어? 소화기 설치 기준은 ...",[기준]
131936,"[마감재, 기타]","뮤럴벽지가 무엇이며, 마감재의 하자여부는 어떻게 판단해?","그림, 벽화라는 의미의 뮤럴벽지는 맞춤벽지라 별도의 제작기간이 필요하다는 특징이 있...","뮤럴벽지가 무엇이며, 마감재의 하자여부는 어떻게 판단해? 그림, 벽화라는 의미의 뮤...","[방법, 기준]"
132032,"[마감재, 기타]","셀룰로오스의 장점이 무엇이고, 소화기 설치 기준이 뭐야?","셀루로오스는 흡음성과 단열성이 좋으며 축열성능이 뛰어나다는 장점이 있습니다. 또한,...","셀룰로오스의 장점이 무엇이고, 소화기 설치 기준이 뭐야? 셀루로오스는 흡음성과 단열...","[장점, 기준]"
132187,"[기타, 마감하자]","층간소음의 기준이 무엇이고, 도배지가 찢어졌어","층간소음의 기준은 1분을 기준으로 주간에는 40dB, 야간에는 3dB을 넘을 경우에...","층간소음의 기준이 무엇이고, 도배지가 찢어졌어 층간소음의 기준은 1분을 기준으로 주...",[기준]


### 추천

In [135]:
mask = train_ft['question'].str.contains('추천|적합|어울릴|가장 좋은')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['추천'])

train_ft[mask]

,category,question,answer,qna,qna_category
62,[마감재],개별 공간과 오픈 플랜 공간 중 어떤 것이 나에게 더 적합한가요?,"개별 공간은 개인 공간과 더 많은 프라이버시를 제공하며, 오픈 플랜은 공간을 확장하...",개별 공간과 오픈 플랜 공간 중 어떤 것이 나에게 더 적합한가요? 개별 공간은 개인...,[추천]
103,[마감재],층간소음을 줄이기 위한 인테리어 추천해줘,"층간소음을 줄이기 위한 인테리어로 카펫, 소음 저감 바닥재 등이 있습니다.","층간소음을 줄이기 위한 인테리어 추천해줘 층간소음을 줄이기 위한 인테리어로 카펫, ...",[추천]
129,[마감재],사우나를 만들려고 해. 어떤 자재가 적합할까?,"사우나를 만들기 위해 적합한 자재는 탄화목, 석재 등이 있습니다. 탄화목, 서재 등...",사우나를 만들려고 해. 어떤 자재가 적합할까? 사우나를 만들기 위해 적합한 자재는 ...,[추천]
441,[시공],공간이 넓게 보이는 인테리어 추천해줘,일반적으로 옅은 색과 세로 패턴의 벽지를 사용하면 공간이 넓어보입니다. 또한 커튼을...,공간이 넓게 보이는 인테리어 추천해줘 일반적으로 옅은 색과 세로 패턴의 벽지를 사용...,[추천]
514,[인테리어],"반려동물을 함께 살 때, 벽지 추천 좀 해줘",반려동물과 함께 거주하는 공간은 긁힘이 발생할 수 있습니다. 따라서 벽면의 긁힘이나...,"반려동물을 함께 살 때, 벽지 추천 좀 해줘 반려동물과 함께 거주하는 공간은 긁힘이...",[추천]
...,...,...,...,...,...
131999,"[마감재, 인테리어]","다공질 흡음재 중 목모보드가 무엇이고, 내 침실을 더 현대적으로 꾸미고 싶어요. 어...",다공질 흡음재 중 목모보드는 목재를 특수 가공하여 만든 것으로 가늘게 채취한 나무 ...,"다공질 흡음재 중 목모보드가 무엇이고, 내 침실을 더 현대적으로 꾸미고 싶어요. 어...",[추천]
132006,"[시공, 인테리어]","배관공사 시 기울기를 주의해야 하는 이유가 무엇이며, 베이 창문을 꾸밀 때 어떤 커...","배관공사 시 기울기를 주지 않으면 물과 이물질이 흐르지 않고 고여서 문제가 되고, ...","배관공사 시 기울기를 주의해야 하는 이유가 무엇이며, 베이 창문을 꾸밀 때 어떤 커...","[주의점, 추천]"
132043,"[마감하자, 마감재]","터짐 하자가 무엇인지, 개별 공간과 오픈 플랜 공간 중 어떤 것이 나에게 더 적합...",도배지가 찢어진 하자는 터짐 하자로 분류됩니다. 터짐 하자가 발생하는 원인과 책임소...,"터짐 하자가 무엇인지, 개별 공간과 오픈 플랜 공간 중 어떤 것이 나에게 더 적합...",[추천]
132047,"[기타, 마감재]","새집증후군의 해결 방법이 무엇인지, 사우나를 만들려고 해. 어떤 자재가 적합할까?","새집증후군을 해결하려면 천연 소재, 포름알데히드 처리를 하지 않은 자재를 사용해야하...","새집증후군의 해결 방법이 무엇인지, 사우나를 만들려고 해. 어떤 자재가 적합할까?...","[대책, 추천]"


### 트렌드

In [136]:
mask = train_ft['question'].str.contains('인기|트렌드|최신')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['트렌드'])

train_ft[mask]

,category,question,answer,qna,qna_category
504,[인테리어],인테리어에 사용되는 도배재료 중에서 가장 인기 있는 소재는 무엇인가요?,현재 가장 인기 있는 도배 소재는 실크벽지입니다. 실크벽지는 다양한 패턴과 색상이 ...,인테리어에 사용되는 도배재료 중에서 가장 인기 있는 소재는 무엇인가요? 현재 가장 ...,[트렌드]
567,[인테리어],벽면에 사용되는 도장재료 중에서 페인트와 타일 중 어떤 것이 더 인기가 있나요?,"페인트와 타일은 각각의 특성에 따라 선택되는데, 페인트는 다양한 색상과 마감 효과를...",벽면에 사용되는 도장재료 중에서 페인트와 타일 중 어떤 것이 더 인기가 있나요? 페...,[트렌드]
580,[인테리어],2023년의 최신 인테리어 디자인 트렌드가 무엇인가요?,"2023년에는 천연 소재, 어스톤 컬러 팔레트, 그리고 지속 가능한 디자인이 트렌드...","2023년의 최신 인테리어 디자인 트렌드가 무엇인가요? 2023년에는 천연 소재, ...",[트렌드]
621,[인테리어],"부엌 리모델링을 고려 중인데, 어떤 타일 디자인이 인기가 있나요?","지금은 대리석, 서브웨이 타일, 그리고 모자이크 패턴의 타일이 인기 있습니다.","부엌 리모델링을 고려 중인데, 어떤 타일 디자인이 인기가 있나요? 지금은 대리석, ...",[트렌드]
1265,[인테리어],인테리어에 사용되는 도배재료 중에서 가장 인기 있는 소재는 무엇인가요?,현재 가장 인기 있는 도배 소재는 실크벽지입니다. 실크벽지는 다양한 패턴과 색상이 ...,인테리어에 사용되는 도배재료 중에서 가장 인기 있는 소재는 무엇인가요? 현재 가장 ...,[트렌드]
...,...,...,...,...,...
131769,"[인테리어, 마감재]","인테리어에 사용되는 도배재료 중에서 가장 인기 있는 소재는 무엇이며, 아크릴 계열 ...",현재 가장 인기 있는 도배 소재는 실크벽지입니다. 실크벽지는 다양한 패턴과 색상이 ...,"인테리어에 사용되는 도배재료 중에서 가장 인기 있는 소재는 무엇이며, 아크릴 계열 ...","[목적.용도, 트렌드]"
131812,"[타마감하자, 인테리어]","천장 부위 결로의 대책은 무엇이고, 벽면에 사용되는 도장재료 중에서 페인트와 타일 ...",천장 부위의 결로를 방지하기 위한 대책으로 실내에서 발생하는 수증기의 양을 최대한 ...,"천장 부위 결로의 대책은 무엇이고, 벽면에 사용되는 도장재료 중에서 페인트와 타일 ...","[대책, 트렌드]"
131973,"[건축구조, 인테리어]","철골콘크리트 구조가 무엇인지, 또 부엌 리모델링을 고려 중인데, 어떤 타일 디자인이...","철근콘크리트는 기둥, 보, 내력벽, 바닥 슬라브 등의 주요 구조부가 철근 콘크리트 ...","철골콘크리트 구조가 무엇인지, 또 부엌 리모델링을 고려 중인데, 어떤 타일 디자인이...",[트렌드]
132041,"[마감재, 인테리어]","굽도리가 무엇이고, 2023년의 최신 인테리어 디자인 트렌드가 무엇인가요?",굽도리란 걸레받이를 대체할 수 있는 자재로 스티커 형태로 벽과 바닥면 이음새에 부착...,"굽도리가 무엇이고, 2023년의 최신 인테리어 디자인 트렌드가 무엇인가요? 굽도리란...",[트렌드]


### 팁

In [137]:
mask = train_ft['question'].str.contains('팁|꿀팁|선택|아이디어|핵심|핵심 포인트|조언|느낌을 내려면|요령| 잘 |효과적으로') & ~train_ft['question'].str.contains('선택할 때')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['팁'])

train_ft[mask]


,category,question,answer,qna,qna_category
325,[마감재],벽지와 도료 중 어떤 것을 선택해야 하나요?,"벽지는 패턴과 텍스처를 추가할 때 좋고, 도료는 단색이며 부드러운 마감을 원할 때 ...",벽지와 도료 중 어떤 것을 선택해야 하나요? 벽지는 패턴과 텍스처를 추가할 때 좋고...,[팁]
503,[인테리어],실내 식물을 효과적으로 가꾸려면 어떤 종류의 식물을 선택해야 하나요?,"실내 식물을 가꾸려면 자연조명이 있는 곳에 맞는 식물, 공기 정화식물, 그리고 효과...",실내 식물을 효과적으로 가꾸려면 어떤 종류의 식물을 선택해야 하나요? 실내 식물을 ...,"[종류, 팁]"
511,[인테리어],인테리어에서 컬러매치에 대한 꿀팁은 무엇인가요?,컬러매치를 위해서는 대비 컬러와 조화 컬러를 활용하는 것이 중요합니다. 주요 색상과...,인테리어에서 컬러매치에 대한 꿀팁은 무엇인가요? 컬러매치를 위해서는 대비 컬러와 조...,[팁]
513,[인테리어],인테리어에서 자연주의적인 스타일을 살려보려면 어떤 디자인을 선택해야 하나요?,자연주의적인 스타일을 살리려면 자연 소재를 활용하고 자연스러운 색상을 선택해야 합니...,인테리어에서 자연주의적인 스타일을 살려보려면 어떤 디자인을 선택해야 하나요? 자연주...,"[주의점, 팁]"
523,[인테리어],베이 창문을 꾸밀 때 어떤 커튼 스타일이 잘 어울릴까요?,"베이 창문을 꾸밀 때는 커튼 레일을 사용하고, 가볍고 부드러운 커튼을 선택해 곡선에...",베이 창문을 꾸밀 때 어떤 커튼 스타일이 잘 어울릴까요? 베이 창문을 꾸밀 때는 커...,"[추천, 팁]"
...,...,...,...,...,...
132127,"[인테리어, 마감재]","인테리어에서 효과적인 수납공간을 만들기 위한 아이디어는 무엇이며, 그라스울의 단점이...","효과적인 수납공간을 만들기 위해서는 가구의 선택과 배치, 벽면을 활용한 수납장 설치...","인테리어에서 효과적인 수납공간을 만들기 위한 아이디어는 무엇이며, 그라스울의 단점이...","[단점, 팁]"
132161,"[인테리어, 시공]","인테리어에서 색상 선택 시 고려해야 할 요소는 무엇이고, 방청도료 도장은 어떻게 작업해?","색상을 선택할 때는 공간의 크기, 채광 상태, 가구의 색상 등을 고려해야 합니다. ...","인테리어에서 색상 선택 시 고려해야 할 요소는 무엇이고, 방청도료 도장은 어떻게 작...","[방법, 팁]"
132190,[마감재],"노출콘크리트의 단점이 무엇인지, 그리고 벽지와 도료 중 어떤 것을 선택해야 하나요?",노출콘크리트는 정확한 공사비 책정이 어렵고 시공 후 보수와 수정이 어려우며 외부 단...,"노출콘크리트의 단점이 무엇인지, 그리고 벽지와 도료 중 어떤 것을 선택해야 하나요?...","[단점, 팁]"
132196,"[마감재, 인테리어]","비닐사이딩이 무엇이고, 인테리어에서 색상 선택 시 고려해야 할 요소는 무엇인가요?",비닐사이딩은 PVC 수지를 이용하여 목재 사이딩처럼 은은한 나무 무늬 모양으로 성형...,"비닐사이딩이 무엇이고, 인테리어에서 색상 선택 시 고려해야 할 요소는 무엇인가요? ...",[팁]


### 설명

In [138]:
mask = train_ft['question'].str.contains('이 뭐야|가 뭐야|은 뭐야|는 뭐야|설명|설명해|설명해줘|대해 알려줘|알려줘|란 무엇|무엇인가요|알려줄래|알려주|알려주시겠어요|알고싶어|알고 싶|무엇을 의미|의미|좀 더 알려|무엇을 하는|무슨 뜻|무엇이나|무엇이죠|무엇입니까') & ~train_ft['question'].str.contains('종류|장점|단점|방법|대책|해결|트렌드|순서|기준|원인|이유|팁|기능|어떤 방식|선택')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['설명'])

train_ft[mask]

,category,question,answer,qna,qna_category
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...,[설명]
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사...",[설명]
5,[건축구조],커튼월이 뭐야?,"커튼월은 건물의 하중을 모두 기중, 들보, 바닥, 지붕으로 지탱하고, 외벽은 하중을...","커튼월이 뭐야? 커튼월은 건물의 하중을 모두 기중, 들보, 바닥, 지붕으로 지탱하고...",[설명]
6,[건축구조],내진구조가 뭐야?,내진구조란 강한 규모의 지진파에도 건축물의 구조나 시설물들이 붕괴되지 않도록 철근 ...,내진구조가 뭐야? 내진구조란 강한 규모의 지진파에도 건축물의 구조나 시설물들이 붕괴...,[설명]
7,[건축구조],중목구조 방식이 뭐야?,중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재로 큰 각재를 사용하여 ...,중목구조 방식이 뭐야? 중목구조는 기본적으로 기중-보 방식과 유사하지만 구조용 목재...,[설명]
...,...,...,...,...,...
132198,"[마감재, 시공]","합지벽지가 무엇이며, 겨울에 도배 주의사항 좀 알려줘","합지벽지란 종이를 2장 이상 겹쳐서 만든 벽지를 의미합니다. 가격은 평당 3,000...","합지벽지가 무엇이며, 겨울에 도배 주의사항 좀 알려줘 합지벽지란 종이를 2장 이상 ...","[주의점, 설명]"
132203,"[건축구조, 인테리어]","커튼월이 무엇인지, 또한, 펫테리어가 뭐야?","커튼월은 건물의 하중을 모두 기중, 들보, 바닥, 지붕으로 지탱하고, 외벽은 하중을...","커튼월이 무엇인지, 또한, 펫테리어가 뭐야? 커튼월은 건물의 하중을 모두 기중, 들...",[설명]
132206,"[기타, 시공]","이산화탄소 소화기가 무엇이며, 액체방수공사가 뭐야?",이산화탄소 소화기는 이산화탄소(CO2)를 높은 압력으로 압축 액화시켜 단단한 철제 ...,"이산화탄소 소화기가 무엇이며, 액체방수공사가 뭐야? 이산화탄소 소화기는 이산화탄소(...",[설명]
132212,"[기타, 마감재]","분말 소화기를 사용할 때 주의할 점이 무엇인지, 또한, 천연페인트의 특징이 뭐야?","분말 소화기를 사용할 때 주의할 점은 압력계의 눈금 위치가 초록색에 있는지 확인, ...","분말 소화기를 사용할 때 주의할 점이 무엇인지, 또한, 천연페인트의 특징이 뭐야? ...","[주의점, 설명]"


### 특징

In [139]:
mask = train_ft['question'].str.contains('특징')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['특징'])

train_ft[mask]

,category,question,answer,qna,qna_category
116,[마감재],유성페인트의 특징이 뭐야?,"유성페인트의 특징은 휘발성으로 냄새가 나고, 건조가 느리며, 철재나 목재에 사용됩니...","유성페인트의 특징이 뭐야? 유성페인트의 특징은 휘발성으로 냄새가 나고, 건조가 느리...","[설명, 특징]"
149,[마감재],수성페인트의 특징이 뭐야?,"수성페인트는 건물외벽이나 콘크리트, 시멘트 벽, 목재, 벽지 등에 주로 사용됩니다....","수성페인트의 특징이 뭐야? 수성페인트는 건물외벽이나 콘크리트, 시멘트 벽, 목재, ...","[설명, 특징]"
318,[마감재],천연페인트의 특징이 뭐야?,천연페인트의 특징은 페인트를 만드는 원료를 자연에서 얻고 자연적으로 분해가 되는 천...,천연페인트의 특징이 뭐야? 천연페인트의 특징은 페인트를 만드는 원료를 자연에서 얻고...,"[설명, 특징]"
655,[마감재],우레탄 계열의 특징 알려줘,"우수한 내구성과 견고함으로, 실내 및 실외에서 사용하기에 적합합니다. 또한 세정이 ...","우레탄 계열의 특징 알려줘 우수한 내구성과 견고함으로, 실내 및 실외에서 사용하기에...","[설명, 특징]"
656,[마감재],에폭시 계열의 특징 알려줘,강도가 뛰어나고 접착력이 우수하여 광택이 강한 표면을 만들 수 있습니다. 또한 내식...,에폭시 계열의 특징 알려줘 강도가 뛰어나고 접착력이 우수하여 광택이 강한 표면을 만...,"[설명, 특징]"
...,...,...,...,...,...
132122,"[마감재, 시공]","천연페인트의 특징이 무엇인지, 또한, 왜 페인트가 발라진 벽에는 도배할 수 없어?",천연페인트의 특징은 페인트를 만드는 원료를 자연에서 얻고 자연적으로 분해가 되는 천...,"천연페인트의 특징이 무엇인지, 또한, 왜 페인트가 발라진 벽에는 도배할 수 없어? ...",[특징]
132158,[마감재],"원목사이딩의 단점이 무엇이고, 에폭시 계열 페인트의 특징이 뭐야?","원목사이딩의 단점은 가격대가 높고 관리가 어려우며 습기에 약해 뒤틀림, 부서짐, 수...","원목사이딩의 단점이 무엇이고, 에폭시 계열 페인트의 특징이 뭐야? 원목사이딩의 단점...","[단점, 특징]"
132201,[마감재],"수성페인트의 특징이 무엇이고, 파벽돌의 단점이 뭐야?","수성페인트는 건물외벽이나 콘크리트, 시멘트 벽, 목재, 벽지 등에 주로 사용됩니다....","수성페인트의 특징이 무엇이고, 파벽돌의 단점이 뭐야? 수성페인트는 건물외벽이나 콘크...","[단점, 특징]"
132204,[마감재],"탄화목사이딩의 장점이 무엇인지, 또 수성페인트의 특징이 뭐야?",탄화목사이딩은 일반 목재보다 내구성이 좋고 관리가 쉬우며 친환경적인 점이 장점입니다...,"탄화목사이딩의 장점이 무엇인지, 또 수성페인트의 특징이 뭐야? 탄화목사이딩은 일반 ...","[장점, 특징]"


### 순서

In [140]:
mask = train_ft['question'].str.contains('순서|절차')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['순서'])

train_ft[mask]

,category,question,answer,qna,qna_category
27,[건축구조],내진설계에서 안정성이 높은 순서대로 말해줘,"내진설계는 면진구조, 제진구조, 내진구조 순으로 안정성이 높습니다.","내진설계에서 안정성이 높은 순서대로 말해줘 내진설계는 면진구조, 제진구조, 내진구조...",[순서]
480,[시공],도배순서는?,도배 순서는 짐 이동 - 박리 작업 - 초배작업 - 벽지 재단 및 풀칠 - 정배 -...,도배순서는? 도배 순서는 짐 이동 - 박리 작업 - 초배작업 - 벽지 재단 및 풀칠...,[순서]
788,[건축구조],내진설계에서 안정성이 높은 순서대로 말해줘,"내진설계에서 안정성이 높은 순서로는 먼저 면진구조, 다음으로 제진구조, 마지막으로 ...",내진설계에서 안정성이 높은 순서대로 말해줘 내진설계에서 안정성이 높은 순서로는 먼저...,[순서]
1241,[시공],도배순서는?,도배 작업의 순서는 다음과 같습니다. 1. 짐 이동 2. 박리 작업 3. 초배 작업...,도배순서는? 도배 작업의 순서는 다음과 같습니다. 1. 짐 이동 2. 박리 작업 3...,[순서]
1432,[건축구조],내진설계에서 안정성이 높은 순서대로 말해줘,내진설계에서 안정성이 높은 순서대로 설명해 드리겠습니다. 먼저 제진구조가 가장 안정...,내진설계에서 안정성이 높은 순서대로 말해줘 내진설계에서 안정성이 높은 순서대로 설명...,[순서]
...,...,...,...,...,...
130884,"[마감하자, 건축구조]","틈새과다 하자의 해결방법이 무엇이며, 내진설계에서 안정성이 높은 순서대로 말해줘",도배지와 몰딩면에 틈새가 있는 것을 틈새과다 하자라고 합니다. 틈새과다 하자의 원인...,"틈새과다 하자의 해결방법이 무엇이며, 내진설계에서 안정성이 높은 순서대로 말해줘 ...","[대책, 순서]"
131127,"[마감재, 시공]","점토벽돌이 무엇이며, 도배순서는?","점토벽돌이란 점토에 모래, 석화 등을 혼합하여 100도 이상의 열에 가열한 벽돌을 ...","점토벽돌이 무엇이며, 도배순서는? 점토벽돌이란 점토에 모래, 석화 등을 혼합하여 1...",[순서]
131503,"[마감재, 시공]","아스팔트슁글의 단점이 무엇이며, 도배순서는?",아스팔트슁글은 강풍에 약하고 단열성이 낮다는 점이 단점입니다. 또 도배 순서는 짐 ...,"아스팔트슁글의 단점이 무엇이며, 도배순서는? 아스팔트슁글은 강풍에 약하고 단열성이 ...","[단점, 순서]"
131549,"[기타, 건축구조]","방수와 발수의 차이가 무엇인지, 그리고 내진설계에서 안정성이 높은 순서대로 말해줘","방수는 제품 표면에 파라핀, 플라스틱이 우레탄 같은 코팅막을 형성해서 제품에 소수성...","방수와 발수의 차이가 무엇인지, 그리고 내진설계에서 안정성이 높은 순서대로 말해줘 ...","[차이, 순서]"


### 가능성

In [141]:
mask = train_ft['question'].str.contains('가능성|수 있어|수 있나요|수도 있을|가능한가') & ~train_ft['question'].str.contains('어떻게')
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['가능성'])

train_ft[mask]

,category,question,answer,qna,qna_category
188,[마감재],실내 흡연을 하면 벽지가 변색될 수 있어?,실내흡연을 하면 벽지가 누렇게 변색될 수 있습니다.,실내 흡연을 하면 벽지가 변색될 수 있어? 실내흡연을 하면 벽지가 누렇게 변색될 수...,[가능성]
330,[마감하자],건조시간 부족으로 인해 도배지 울음이 발생할 수 있어?,도배 후 완전 건조되기 전까지 울음이 발생할 수 있습니다. 제조사 권장 건조시간(일...,건조시간 부족으로 인해 도배지 울음이 발생할 수 있어? 도배 후 완전 건조되기 전까...,"[기간, 가능성]"
331,[마감하자],절단 부위가 과소하면 도배지가 찢어질 수 있어?,도배지 절단 부위가 구조물에 비해 과소하게 절단되면 장력을 이기지 못하고 찢어질 수...,절단 부위가 과소하면 도배지가 찢어질 수 있어? 도배지 절단 부위가 구조물에 비해 ...,[가능성]
332,[마감하자],기존 실크 벽지를 불완전 제거할 경우 도배지가 찢어질 수 있어?,기존 실크 벽지를 제대로 제거하지 않은 경우 터짐 하자가 발생할 수 있습니다.,기존 실크 벽지를 불완전 제거할 경우 도배지가 찢어질 수 있어? 기존 실크 벽지를 ...,[가능성]
333,[마감하자],누수에 의해 도배지에 녹이 묻어나올 수 있어?,누수에 의해 도배지가 젖어 있는 상태가 지속되면 곰팡이가 발생할 수 있습니다.,누수에 의해 도배지에 녹이 묻어나올 수 있어? 누수에 의해 도배지가 젖어 있는 상태...,[가능성]
...,...,...,...,...,...
132113,"[마감재, 마감하자]","송판 노출콘크리트가 무엇인지, 또한, 내외부 온도차가 클 경우 도배지 끝부분이 들뜰...",송판 노출콘크리트란 나무무늬의 질감이 나는 콘크리트 외장재 마감을 의미합니다. 또한...,"송판 노출콘크리트가 무엇인지, 또한, 내외부 온도차가 클 경우 도배지 끝부분이 들뜰...",[가능성]
132163,"[건축구조, 마감하자]","기둥-보 구조 방식이 무엇이며, 구조적 결함으로 인해 석고수정이 발생할 수 있어?",기둥-보 구조 방식은 가장 오래된 목구조 방식 중 하나로 통나무 구조에서 발전된 건...,"기둥-보 구조 방식이 무엇이며, 구조적 결함으로 인해 석고수정이 발생할 수 있어? ...",[가능성]
132191,"[마감재, 마감하자]","온돌마루의 장점이 무엇이며, 절단 부위가 과소하면 도배지가 찢어질 수 있어?",온돌마루는 강마루처럼 열전도율이 높고 보행감이 좋으며 원목의 질감을 느낄 수 있는 ...,"온돌마루의 장점이 무엇이며, 절단 부위가 과소하면 도배지가 찢어질 수 있어? 온돌마...","[장점, 가능성]"
132202,"[마감재, 마감하자]","방부목의 장점이 무엇인지, 또한, 내외부 온도의 큰 차이에 의해 곰팡이가 발생할 수...","방부목은 데크재 중 가장 저렴하다는 장점입니다. 또한, 내외부 온도차가 클 경우 도...","방부목의 장점이 무엇인지, 또한, 내외부 온도의 큰 차이에 의해 곰팡이가 발생할 수...","[장점, 가능성]"


### 이유

In [142]:
mask = train_ft['question'].str.contains('이유|왜')  # 어떤 것이 있어?

train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['이유'])
train_ft[mask]

,category,question,answer,qna,qna_category
169,[마감재],석고보드를 사용하는 이유가 뭐야?,석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,석고보드를 사용하는 이유가 뭐야? 석고보드는 인체에 무해하며 친환경적이므로 마감재로...,"[목적.용도, 이유]"
394,[시공],왜 초배지만 남은 벽에는 페인트를 못 칠해?,초배지는 매우 얇기 얇아 페인트를 바로 칠하면 표면이 쭈글쭈글하게 울기에 초배지 위...,왜 초배지만 남은 벽에는 페인트를 못 칠해? 초배지는 매우 얇기 얇아 페인트를 바로...,[이유]
407,[시공],왜 페인트가 발라진 벽에는 도배할 수 없어?,기존에 페인트가 칠해진 벽은 벽지가 잘 붙지 않고 떨어져 하자가 발생할 확률이 높기...,왜 페인트가 발라진 벽에는 도배할 수 없어? 기존에 페인트가 칠해진 벽은 벽지가 잘...,[이유]
410,[시공],왜 실크벽지가 합지 벽지에 비해 필요한 도배사가 더 많아?,실크벽지의 경우 합지벽지와 달리 초배작업이 필요하므로 도배사가 필요합니다.,왜 실크벽지가 합지 벽지에 비해 필요한 도배사가 더 많아? 실크벽지의 경우 합지벽지...,"[비교, 이유]"
458,[시공],배관공사 시 기울기를 주의해야 하는 이유가 뭐야?,"배관공사 시 기울기를 주지 않으면 물과 이물질이 흐르지 않고 고여서 문제가 되고, ...",배관공사 시 기울기를 주의해야 하는 이유가 뭐야? 배관공사 시 기울기를 주지 않으면...,"[주의점, 이유]"
...,...,...,...,...,...
132104,"[마감재, 시공]","석고보드를 사용하는 이유가 무엇인지, 또한, 초배작업이 뭐야?",석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,"석고보드를 사용하는 이유가 무엇인지, 또한, 초배작업이 뭐야? 석고보드는 인체에 무...","[목적.용도, 이유]"
132122,"[마감재, 시공]","천연페인트의 특징이 무엇인지, 또한, 왜 페인트가 발라진 벽에는 도배할 수 없어?",천연페인트의 특징은 페인트를 만드는 원료를 자연에서 얻고 자연적으로 분해가 되는 천...,"천연페인트의 특징이 무엇인지, 또한, 왜 페인트가 발라진 벽에는 도배할 수 없어? ...","[특징, 이유]"
132141,"[인테리어, 마감재]","반려동물과 함께 살 때, 바닥에서 1미터 높이에 몰딩을 설치하는 것을 추천하는 이유...",바닥에서 1미터 높이에 몰딩을 설치하면 하단부의 벽지가 훼손될 때마다 하단부의 벽지...,"반려동물과 함께 살 때, 바닥에서 1미터 높이에 몰딩을 설치하는 것을 추천하는 이유...","[장점, 추천, 이유]"
132162,[마감재],"석고보드를 사용하는 이유가 무엇인지, 또한, 차음재 종류는 뭐가 있어?",석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,"석고보드를 사용하는 이유가 무엇인지, 또한, 차음재 종류는 뭐가 있어? 석고보드는 ...","[종류, 목적.용도, 이유]"


### 확인

In [143]:
mask = train_ft['question'].str.contains('해도 돼?|버려도돼?|문제가 없을까|해도 되나요|해도 될까')  # 어떤 것이 있어?

train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['확인'])
train_ft[mask]


,category,question,answer,qna,qna_category
175,[마감재],남은 도배풀을 변기에 버려도 돼?,도배풀에 본드가 섞여있으면 배관을 막을 수도 있으니 일반 종량제 봉투에 버리는 것을...,남은 도배풀을 변기에 버려도 돼? 도배풀에 본드가 섞여있으면 배관을 막을 수도 있으...,[확인]
397,[시공],기존 바닥재 위에 새 바닥재를 덧방해도 되나요?,기존 바닥재를 철거 후 시공해야 합니다. 기존 바닥재 위해 덧방할 경우 바닥재 간에...,기존 바닥재 위에 새 바닥재를 덧방해도 되나요? 기존 바닥재를 철거 후 시공해야 합...,[확인]
448,[시공],외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?,외부에 소리가 안 나가게 하는 목적으로 차음재만 설치할 경우 소리를 차단하고 반사하...,외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼? 외부에 소리가 안 나가...,[확인]
452,[시공],도배 후 필름 시공해도 될까?,일반적으로 필름 시공 후 도배를 합니다. 그 이유는 벽지 위에 붙은 필름의 부착력이...,도배 후 필름 시공해도 될까? 일반적으로 필름 시공 후 도배를 합니다. 그 이유는 ...,[확인]
495,[시공],기존 벽지를 제거하지 않고 도배해도 될까?,도배지에 따라 다릅니다. 일반적으로 합지 위에 합지를 도배하는 것은 가능합니다. 하...,기존 벽지를 제거하지 않고 도배해도 될까? 도배지에 따라 다릅니다. 일반적으로 합지...,[확인]
...,...,...,...,...,...
131571,"[인테리어, 시공]","실내 공간에 자연적인 감각을 불어넣기 위한 방법이 무엇이며, 도배 후 필름 시공해도...","실내 공간에 자연적인 감각을 추가하려면 나무 가구, 식물, 그리고 자연적인 소재를 ...","실내 공간에 자연적인 감각을 불어넣기 위한 방법이 무엇이며, 도배 후 필름 시공해도...","[방법, 확인]"
131673,"[마감재, 시공]","알루미늄징크의 단점은 무엇이고, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치...",알루미늄징크은 시공이 잘못될 경우 우는 현상이 발생하고 리얼징크보다 가격이 좀 더 ...,"알루미늄징크의 단점은 무엇이고, 외부에 소리가 안나가게 하는 목적으로 차음재만 설치...","[단점, 확인]"
132000,"[타마감하자, 시공]","결로가 발생하는 원인이 무엇이고, 도배 후 필름 시공해도 될까?","결로가 발생하는 원인은 높은 습도와 차가운 표면온도가 있습니다. 또한, 일반적으로 ...","결로가 발생하는 원인이 무엇이고, 도배 후 필름 시공해도 될까? 결로가 발생하는 원...","[원인, 확인]"
132119,"[마감재, 시공]","페인트가 무엇인지, 또 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?","페인트란 소지의 외관에 도막을 형성함으로써, 각종 원자재나 완제품의 노화, 산화 등...","페인트가 무엇인지, 또 외부에 소리가 안나가게 하는 목적으로 차음재만 설치해도 돼?...",[확인]


### 상황

In [144]:
mask = train_ft['question'].str.contains('생겼어|찢어졌어|들떠있어|틈새가 있어|얼룩이 있어|이어지지 않아|묻어나와|폈어|이상해')  # 어떤 것이 있어?

train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['상황'])
train_ft[mask]


,category,question,answer,qna,qna_category
334,[마감하자],도배지에 울음이 생겼어,"울음이란 도배지가 볼록하게 튀어나오는 현상으로, 건조시간 부족이 원인일 수 있습니다...","도배지에 울음이 생겼어 울음이란 도배지가 볼록하게 튀어나오는 현상으로, 건조시간 부...",[상황]
343,[마감하자],도배지와 몰딩면에 틈새가 있어,도배지와 몰딩면에 틈새가 있는 하자를 틈새과다 하자라고 합니다. 틈새과다 하자 발생...,도배지와 몰딩면에 틈새가 있어 도배지와 몰딩면에 틈새가 있는 하자를 틈새과다 하자라...,[상황]
348,[마감하자],도배지가 찢어졌어,"도배지가 찢어진 경우 발생하는 원인, 책임소재, 해결방법에 대해 설명드리겠습니다. ...","도배지가 찢어졌어 도배지가 찢어진 경우 발생하는 원인, 책임소재, 해결방법에 대해 ...",[상황]
357,[마감하자],도배지가 끝부분이 들떠있어,도배지 들뜸 하자는 도배지 끝 부분이 벽면에 부착되지 않는 현상입니다. 들뜸 하자는...,도배지가 끝부분이 들떠있어 도배지 들뜸 하자는 도배지 끝 부분이 벽면에 부착되지 않...,[상황]
367,[마감하자],도배지 무늬의 패턴이 이어지지 않아,도배지 무늬의 패턴이 끊기는 이음부 하자는 시공자의 부주의나 공간 내 높은 습도로 ...,도배지 무늬의 패턴이 이어지지 않아 도배지 무늬의 패턴이 끊기는 이음부 하자는 시공...,[상황]
...,...,...,...,...,...
131702,"[마감재, 마감하자]","수성페인트의 특징이 무엇이고, 도배지에 녹이 묻어나와","수성페인트는 건물외벽이나 콘크리트, 시멘트 벽, 목재, 벽지 등에 주로 사용됩니다....","수성페인트의 특징이 무엇이고, 도배지에 녹이 묻어나와 수성페인트는 건물외벽이나 콘크...","[특징, 상황]"
131728,"[기타, 마감하자]","새집증후군의 해결 방법이 무엇인지, 도배지에 울음이 생겼어","새집증후군을 해결하려면 천연 소재, 포름알데히드 처리를 하지 않은 자재를 사용해야하...","새집증후군의 해결 방법이 무엇인지, 도배지에 울음이 생겼어 새집증후군을 해결하려면...","[대책, 상황]"
131983,"[인테리어, 마감하자]","거미식물의 효과는 무엇인지, 또한, 도배지에 반점이 생겼어","거미식물의 효과는 공기 중 독성물질(일산화탄소, 포름알데히드, 자일렌, 톨루엔 등)...","거미식물의 효과는 무엇인지, 또한, 도배지에 반점이 생겼어 거미식물의 효과는 공기 ...","[기능.효과, 상황]"
132176,"[시공, 마감하자]","장롱 옮길 때 주의할 점이 무엇이고, 도배지에 반점이 생겼어",장롱을 옮길 때 주의할 점은 장판이 상하지 않도록 헝겁 등의 천을 대는 것이 좋습니...,"장롱 옮길 때 주의할 점이 무엇이고, 도배지에 반점이 생겼어 장롱을 옮길 때 주의할...","[주의점, 상황]"


### 기타 질문

In [145]:
mask = train_ft['qna_category'].apply(lambda x: len(x) == 0)
train_ft.loc[mask, 'qna_category'] = train_ft.loc[mask, 'qna_category'].apply(lambda x: x + ['기타질문'])

train_ft[mask]

,category,question,answer,qna,qna_category
110,[마감재],흡음재 구조는 어떤 것이 있어?,"흡음재의 구조로 다공질, 공명형, 판 진동형이 있습니다.","흡음재 구조는 어떤 것이 있어? 흡음재의 구조로 다공질, 공명형, 판 진동형이 있습니다.",[기타질문]
208,[마감재],알러지 있는 사람들은 어떤 벽지 쓰는게 좋아?,알러지가 있는 사람은 표면이 먼지나 알러지 유발물질이 표면에 적게 축적되는 매끄럽고...,알러지 있는 사람들은 어떤 벽지 쓰는게 좋아? 알러지가 있는 사람은 표면이 먼지나 ...,[기타질문]
255,[마감재],남은 도배풀은 어디에 버려?,남은 도배풀은 일반 종량제 봉투에 버리셔야 합니다.,남은 도배풀은 어디에 버려? 남은 도배풀은 일반 종량제 봉투에 버리셔야 합니다.,[기타질문]
379,[마감하자],곰팡이 색깔은?,"도배지에 나타나는 곰팡이의 색갈은 검정색, 황갈색, 분홍색 등이 있습니다.","곰팡이 색깔은? 도배지에 나타나는 곰팡이의 색갈은 검정색, 황갈색, 분홍색 등이 있...",[기타질문]
383,[마감하자],벽지 안쪽의 못 등의 작은 피스로 인해 발생하는 하자는 무슨 하자야?,벽지 안쪽의 못 등의 작은 피스로 인해 발생하는 하자는 피스 하자로 심할 경우 도배...,벽지 안쪽의 못 등의 작은 피스로 인해 발생하는 하자는 무슨 하자야? 벽지 안쪽의 ...,[기타질문]
...,...,...,...,...,...
131793,"[기타, 인테리어]","분말 소화기가 무엇이며, 인테리어에서 산업 스타일을 표현하려면 어떤 디자인을 적용해...",분말 소화기는 탄산수소 나트륨 분말을 이용해 질소나 이산화탄소 등 불연성 고압가스에...,"분말 소화기가 무엇이며, 인테리어에서 산업 스타일을 표현하려면 어떤 디자인을 적용해...",[기타질문]
131797,"[마감재, 인테리어]","청고벽돌이 무엇인지, 또한, 인테리어에서 창문 커버링을 선택할 때 고려해야 할 사항...","청고벽돌은 회색빛과 청색빛이 도는 고벽돌을 의미하며, 이전에 지어진 건물에 사용됐던...","청고벽돌이 무엇인지, 또한, 인테리어에서 창문 커버링을 선택할 때 고려해야 할 사항...",[기타질문]
131811,"[마감재, 시공]","방염벽지가 무엇이고, 방음공사 할 경우 보통 차음재와 흡음재는 각각 어디에 넣어",방염벽지는 실크벽지와 같이 PVC 코팅을 하고 방염처리가 된 벽지입니다. 가격은 평...,"방염벽지가 무엇이고, 방음공사 할 경우 보통 차음재와 흡음재는 각각 어디에 넣어 방...",[기타질문]
131930,"[마감재, 인테리어]","역청질페인트가 무엇이며, 인테리어에서 산업 스타일을 표현하려면 어떤 디자인을 적용해...","역청질페인트란 아스팔트, 타르핏치 등의 역청질을 주원료로 하여 건성유, 수지류를 첨...","역청질페인트가 무엇이며, 인테리어에서 산업 스타일을 표현하려면 어떤 디자인을 적용해...",[기타질문]


# 카테고리 완료

In [146]:
train_ft

,category,question,answer,qna,qna_category
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...,[설명]
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.",[종류]
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...,[장점]
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사...",[설명]
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.,[종류]
...,...,...,...,...,...
132215,"[건축구조, 마감재]","경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ...","[장점, 방법]"
132216,[마감재],"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나...","[단점, 기간]"
132217,[마감재],"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘",모노롱브릭타일은 일반 시멘트벽돌과는 다르게 얇고 긴 모양의 타일 형태로 와이드 벽돌...,"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘 모노롱브릭타일은 일반 시멘트벽돌과...",[설명]
132218,[마감재],"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로...","[단점, 방법]"


In [147]:
mask = train_ft['qna_category'].apply(lambda x : len(x) == 3)
train_ft[mask]

,category,question,answer,qna,qna_category
758,[마감하자],도배지 반점을 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 방법의 장단점...,반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 방법의 장점은 상대적으로 간...,도배지 반점을 반점을 바인더나 수성 프라이머로 코팅하고 다시 도배하는 방법의 장단점...,"[장점, 단점, 방법]"
759,[마감하자],도배지 반점이 생긴 부분의 석고보드를 부분적으로 잘라내고 보강 후 재작업하는 방법의...,반점이 생긴 부분의 석고보드를 부분적으로 잘라내고 보강 후 재작업하는 방법의 장점은...,도배지 반점이 생긴 부분의 석고보드를 부분적으로 잘라내고 보강 후 재작업하는 방법의...,"[장점, 단점, 방법]"
760,[마감하자],"벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장단점이 뭐야?","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장점은, 가장...","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장단점이 뭐야...","[장점, 단점, 방법]"
6560,[건축구조],철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,방염벽지는 내구성과 내스크래치성이 우수하고 화재를 대비할 때 사용하기 좋다는 장점이...,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요? 방염벽지는 내구성과 내스...,"[장점, 단점, 종류]"
6598,[기타],공동주택의 실내공기질을 측정하기 위한 지점 설정은 어떻게 하는 게 좋을까요?,라돈을 제외한 공동주택의 실내공기질 측정시료는 실내공기질공정시험기준에 의하여 100...,공동주택의 실내공기질을 측정하기 위한 지점 설정은 어떻게 하는 게 좋을까요? 라돈을...,"[방법, 계산, 기준]"
...,...,...,...,...,...
132141,"[인테리어, 마감재]","반려동물과 함께 살 때, 바닥에서 1미터 높이에 몰딩을 설치하는 것을 추천하는 이유...",바닥에서 1미터 높이에 몰딩을 설치하면 하단부의 벽지가 훼손될 때마다 하단부의 벽지...,"반려동물과 함께 살 때, 바닥에서 1미터 높이에 몰딩을 설치하는 것을 추천하는 이유...","[장점, 추천, 이유]"
132157,"[마감재, 마감하자]","초경벽지가 무엇인지, 또한, 도배지 반점이 생긴 부분의 석고보드를 부분적으로 잘라내...","천연벽지에 속하는 초경벽지는 칡, 대나무, 갈대 등 식물을 그대로 가공하여 원지에 ...","초경벽지가 무엇인지, 또한, 도배지 반점이 생긴 부분의 석고보드를 부분적으로 잘라내...","[장점, 단점, 방법]"
132162,[마감재],"석고보드를 사용하는 이유가 무엇인지, 또한, 차음재 종류는 뭐가 있어?",석고보드는 인체에 무해하며 친환경적이므로 마감재로 많이 사용합니다. 특히 가볍고 절...,"석고보드를 사용하는 이유가 무엇인지, 또한, 차음재 종류는 뭐가 있어? 석고보드는 ...","[종류, 목적.용도, 이유]"
132205,"[시공, 마감재]","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 흡음재 중 다공질은 뭐...","내단열 시공은 외단열 시공에 비해 시공 난이도가 낮으며 상대적으로 공사비, 재료비,...","내단열 시공이 외단열 시공에 비해 갖는 장점이 무엇인지, 또 흡음재 중 다공질은 뭐...","[장점, 종류, 비교]"


In [148]:
mask = train_ft['qna_category'].apply(lambda x : len(x) >3)
train_ft[mask]

,category,question,answer,qna,qna_category
19469,[기타],공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요?,공동주택의 실내 라돈은 최저층에서 측정하며 공동주택의 총 세대수가 100세대일 때 ...,공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요? 공동주택...,"[방법, 계산, 기준, 추천]"
20113,[기타],공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요?,"라돈은 자연적으로 발생하는 기체 상태의 방사성 물질로, 건물 내부에서 발생하는 물질...",공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요? 라돈은 ...,"[방법, 계산, 기준, 추천]"
20757,[기타],공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요?,"라돈은 공동주택의 최저층에서 측정하며, 공동주택의 총 세대수에 따라 측정 지점을 결...",공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요? 라돈은 ...,"[방법, 계산, 기준, 추천]"
21401,[기타],공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요?,"라돈은 지반과 암석 등에서 나오는 강심도 자연 방사성 물질인데, 공동주택의 실내 라...",공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요? 라돈은 ...,"[방법, 계산, 기준, 추천]"
22045,[기타],공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요?,"라돈은 밀봉된 병에 채취되며, 공동주택의 최저층에 위치한 세대에서 측정됩니다. 공동...",공동주택에서 가장 적합한 곳에 라돈 측정지점을 설정하는 방법이 무엇인가요? 라돈은 ...,"[방법, 계산, 기준, 추천]"
...,...,...,...,...,...
131165,"[마감하자, 마감재]","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장단점이 무엇...","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장점은, 가장...","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장단점이 무엇...","[장점, 단점, 비교, 방법]"
131331,"[시공, 인테리어]","배관공사 시 기울기를 주의해야 하는 이유가 무엇이며, 실내 식물을 효과적으로 가꾸려...","배관공사 시 기울기를 주지 않으면 물과 이물질이 흐르지 않고 고여서 문제가 되고, ...","배관공사 시 기울기를 주의해야 하는 이유가 무엇이며, 실내 식물을 효과적으로 가꾸려...","[종류, 주의점, 팁, 이유]"
131384,"[마감하자, 인테리어]","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장단점이 무엇...","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장점은, 가장...","벽면 전체 석고보드를 잘라내고, 석고보드를 교체하여 재작업하는 방법의 장단점이 무엇...","[장점, 단점, 방법, 팁]"
131422,"[기타, 인테리어]","새집증후군을 예방하는 방법이 무엇인지, 또 베이 창문을 꾸밀 때 어떤 커튼 스타일이...",새집증후군을 예방하는 방법은 실내흡연을 하지않고 포르말린을 사용하지 않은 목재 가구...,"새집증후군을 예방하는 방법이 무엇인지, 또 베이 창문을 꾸밀 때 어떤 커튼 스타일이...","[방법, 대책, 추천, 팁]"


In [149]:
train_ft

,category,question,answer,qna,qna_category
0,[건축구조],면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,면진장치가 뭐야? 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동...,[설명]
1,[건축구조],내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계의 종류 좀 알려줘 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.",[종류]
2,[건축구조],철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점이 뭐야? 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문...,[장점]
3,[건축구조],철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사용하여 강성과 내구성을 높이는 구...","철골철근 콘크리트 구조가 뭐야? 철근철골콘크리트는 철골과 철근, 콘크리트를 함께 사...",[설명]
4,[건축구조],철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 어떤 방식이 있어? 철골구조는 일반철골구조와 경량철골구조가 있습니다.,[종류]
...,...,...,...,...,...
132215,"[건축구조, 마감재]","경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야?",경량목구조 방식은 투바이포 공법이라고도 불리며 2x4(inch)의 각재를 사용하여 ...,"경량목구조 방식이 무엇이고, 비드법 보온판 2종의 장점이 뭐야? 경량목구조 방식은 ...","[장점, 방법]"
132216,[마감재],"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야?",아이소핑크는 시간이 지나면 단열 성능이 떨어지는 경시현상이 있고 비드법 단열재에 비...,"아이소핑크의 단점이 무엇이고, 카페트는 기대수명이 얼마야? 아이소핑크는 시간이 지나...","[단점, 기간]"
132217,[마감재],"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘",모노롱브릭타일은 일반 시멘트벽돌과는 다르게 얇고 긴 모양의 타일 형태로 와이드 벽돌...,"롱브릭타일은 무엇인지, 또 강화마루에 대해 알려줘 모노롱브릭타일은 일반 시멘트벽돌과...",[설명]
132218,[마감재],"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘",셀룰로오스는 목재로부터 만들어지는 종이를 재활용하여 난연재를 첨가해 제조한 것을 의...,"셀룰로오스가 무엇이며, 비드법 보온판 2종의 단점에 대해 알려줘 셀룰로오스는 목재로...","[단점, 방법]"


In [150]:
train_ft.to_csv(f'{data_path}category_train_132220_ver1.csv', index=False)

In [151]:
# train_ft.to_csv(f'{data_path}category train_ft_237983.csv', index=False)

In [152]:
# train_ft.to_csv(f'{data_path}category_train_80000_ver3.csv', index=False)

In [153]:
# train_ft.to_csv(f'{data_path}category_train_ft_97983.csv', index=False)